In [1]:
import pandas as pd
import numpy as np

### Get all the pillar names from the excel

In [2]:
names = pd.read_excel('../../UNDP Digital Assessment Data Framework Filename Matching V7.xlsx')

In [3]:
col_names = ['Indicator','check', 'Data Source','Data Link','Raw/Index','Filename','Sub-Pillar']

In [4]:
names = names[col_names]

In [5]:
names.head()

Indicator           check  \
0                                          Countries             NaN   
1  Database of Global Administrative Areas (GADM,...             NaN   
2  High Resolution Population Density Maps + Demo...             NaN   
3  population density vs openstreetmap object den...             NaN   
4                                 Population Density  Infrastructure   

                                 Data Source  \
0  UN Statistics Division: List of Countries   
1                                        NaN   
2                                        NaN   
3                                        NaN   
4   World Bank: World Development Indicators   

                           Data Link Raw/Index            Filename  \
0             https://unstats.un.org       Raw           Countries   
1                   https://gadm.org       Raw                 NaN   
2                                NaN       NaN                 NaN   
3                                NaN       NaN                 NaN   
4  https://datacatalog.worldbank.org       Raw  population_density   

                Sub-Pillar  
0                      NaN  
1                      NaN  
2                      NaN  
3                      NaN  
4  Connectivity Technology

In [6]:
# get all the files per pillar
data_stats = names.groupby('check').agg({'Filename':'count','Indicator':'count'})

In [7]:
data_stats

Filename  Indicator
check                              
Business              19         28
Foundations           13         22
Government            11         16
Infrastructure        47         58
People                35         49
Regulation             5          8
Strategy               1          1

### Business

In [8]:
bnames = names[(names.check=='Business')&(~names.Filename.isna())]#&(names.Index==False)]

In [9]:
bnames.head(25)

Indicator     check  \
87   UNCTAD Business-to-Consumer (B2C) E-commerce I...  Business   
88       Networking Services (Spend, IT Forecast Data)  Business   
93            Cloud Services (Spend, IT Forecast Data)  Business   
94   ICT task-intensive jobs as a percentage of tot...  Business   
97                     Share of business with internet  Business   
98                  Share of businesses with broadband  Business   
99            Share of businesses with online presence  Business   
100                     Size of gig economy (% of GDP)  Business   
101        Size of digital economy (% of transactions)  Business   
102                       Venture Capital Availability  Business   
103                               Doing Business Index  Business   
105                           Strength of Legal Rights  Business   
106                           Time to Start a Business  Business   
107                             Ease of Doing Business  Business   
110                  Ease of finding skilled employees  Business   
111  Amount invested into startups yearly from priv...  Business   
112                                 Regulatory Quality  Business   
113                             Ease of Getting Credit  Business   
114                            Global Resilience Index  Business   

                                           Data Source  \
87   UNCTAD:  Business-to-Consumer (B2C) E-commerce...   
88        Portulans Institute: Network Readiness Index   
93                                            Statista   
94                         OECD: Going Digital Toolkit   
97            OECD: ICT Access and Usage by Businesses   
98                         OECD: Innovation Indicators   
99        Portulans Institute: Network Readiness Index   
100       Portulans Institute: Network Readiness Index   
101       Portulans Institute: Network Readiness Index   
102  World Economic Forum: Global Competitiveness I...   
103                         World Bank: Doing Business   
105           World Bank: World Development Indicators   
106           World Bank: World Development Indicators   
107                World Bank: Doing Business rankings   
110                                World Bank: GovData   
111                 OECD: Venture capital investments    
112  Global Innovation Index/World Bank: Regulatory...   
113  Global Innovation Index/World Bank: Ease of Ge...   
114              FM Global: FM Global Resilience Index   

                                             Data Link Raw/Index  \
87                                  https://unctad.org     Index   
88                   https://networkreadinessindex.org     Index   
93                            https://www.statista.com     Index   
94                        http://goingdigital.oecd.org     Index   
97                              https://stats.oecd.org     Index   
98   https://www.oecd.org/innovation/inno/inno-stat...     Index   
99                   https://networkreadinessindex.org     Index   
100                  https://networkreadinessindex.org     Index   
101                  https://networkreadinessindex.org     Index   
102                         http://reports.weforum.org     Index   
103                  https://datacatalog.worldbank.org     Index   
105                  https://datacatalog.worldbank.org     Index   
106                  https://datacatalog.worldbank.org       Raw   
107                  https://datacatalog.worldbank.org       NaN   
110                  https://datacatalog.worldbank.org     Index   
111                             https://stats.oecd.org       Raw   
112  https://www.globalinnovationindex.org/analysis...     Index   
113  https://www.globalinnovationindex.org/analysis...     Index   
114                          https://www.fmglobal.com/     Index   

                                Filename            Sub-Pillar  
87                     b2c_ecommerse_idx   Technology Adoption  
88               network_readine

In [10]:
# get list of names for all indicators
indicators = bnames.Indicator.unique()
subpillars = bnames['Sub-Pillar'].unique()

In [11]:
# get all file names
bfiles = bnames.Filename.unique()

In [12]:
bfiles

array(['b2c_ecommerse_idx', 'network_readiness_index', 'cloud_services',
       'ICT_proportion', 'business_internet', 'business_broadband',
       'share_of_businesses_online_presence', 'prevalance_gig_economy',
       'size_digital_economy', 'TCdata360', 'doing_bus_idx',
       'legal_rights_strength', 'time_start_bus', 'ease_doing_bus',
       'ease_of_finding_skilled_employees', 'start_up_investment',
       'global_innovation_dataset', 'global_resilience_index'],
      dtype=object)

In [13]:
subpillars

array(['Technology Adoption', 'Financing Incentives',
       'Startup Environment'], dtype=object)

In [14]:
# ls digital-readiness-assessment-main/processed/

In [15]:
# formula for converting scale 0-100
def convert_rank(old_value, old_min=min, old_max=max, new_min=1, new_max=5.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [16]:
# formula for converting scale 1-7
def convert_rank_b(old_value, old_min=1, old_max=7, new_min=1, new_max=5.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [17]:
# formula for converting scale 0-12
def convert_rank_c(old_value, old_min=0, old_max=12, new_min=1, new_max=5.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [18]:
# formula for converting scale quintile
def convert_rank_I(old_value, old_min,old_max,new_min=1, new_max=1.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value
def convert_rank_II(old_value, old_min, old_max, new_min=2, new_max=2.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value
def convert_rank_III(old_value, old_min, old_max, new_min=3, new_max=3.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value
def convert_rank_IV(old_value, old_min, old_max, new_min=4, new_max=4.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value
def convert_rank_V(old_value, old_min, old_max, new_min=5, new_max=5.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [19]:
### Add population dataset
df = pd.read_csv('../../processed/Population.csv')

# Normalize country names as much as possible
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bahamas.*$)', 'Bahamas (the)')
df['Country Name'] = df['Country Name'].replace('Bahrain (Kingdom of)','Bahrain')
df['Country Name'] = df['Country Name'].replace('Bolivia','Bolivia (Plurinational State of)')
df['Country Name'] = df['Country Name'].replace('Bolivia, Plurinational State of','Bolivia (Plurinational State of)')
df['Country Name'] = df['Country Name'].replace('Brunei','Brunei Darussalam')
df['Country Name'] = df['Country Name'].replace('Bulgaria (Rep.)','Bulgaria')
df['Country Name'] = df['Country Name'].replace('Central African Republic','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace('Central African Rep.','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace("China (People's Rep.)",'China')
df['Country Name'] = df['Country Name'].replace("Comoros",'Comoros (the)')
df['Country Name'] = df['Country Name'].replace("Congo",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Brazzaville)",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Rep. of the)",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Democratic Republic of the)",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo, Dem. Rep.",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo, The Democratic Republic of the",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("DR Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Democratic Republic of Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Democratic Republic of the Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Dem. Rep. of the Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Cote d'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Côte d’Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cote D'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cote dIvoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].str.replace(r"(^.*Côte d'Ivoire.*$)", "Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cōte d'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Ivory Coast","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Dem. People's Rep. of Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("Democratic People's Republic of Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("Korea, Dem. People's Rep.","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("North Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Czech.*$)', 'Czechia')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Dominican Re.*$)', 'Dominican Republic (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Hong Kong.*$)', 'China, Hong Kong Special Administrative Region')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Hongkong.*$)', 'China, Hong Kong Special Administrative Region')
df['Country Name'] = df['Country Name'].replace("Eswatini (Kingdom of)",'Eswatini')
df['Country Name'] = df['Country Name'].replace("Swaziland",'Eswatini')
df['Country Name'] = df['Country Name'].replace("Faröe Islands",'Faroe Islands')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Gambia.*$)', 'Gambia (the)')
df['Country Name'] = df['Country Name'].replace("Georgia (Country)",'Georgia')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Iran.*$)', 'Iran (Islamic Republic of)')
df['Country Name'] = df['Country Name'].replace("Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Rep. of)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Rep.)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Republic of)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Korea, Rep.*$)', 'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea, South",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("South Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Republic of Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace('Republic of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kyrgyz.*$)', 'Kyrgyzstan')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Lao.*$)', "Lao People's Democratic Republic (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macao.*$)', "China, Macao Special Administrative Region")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macau.*$)', "China, Macao Special Administrative Region")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Micronesia.*$)', "Micronesia (Federated States of)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Moldova.*$)', "Republic of Moldova (the)")
df['Country Name'] = df['Country Name'].replace("Morroco",'Morocco')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Nepal.*$)', "Nepal")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*New Ze.*$)', "New Zealand")
df['Country Name'] = df['Country Name'].replace("Niger",'Niger (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macedonia.*$)', "North Macedonia")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*New Ze.*$)', "New Zealand")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Palestin.*$)', "State of Palestine (the)")
df['Country Name'] = df['Country Name'].replace("West Bank and Gaza",'State of Palestine (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Panama.*$)', "Panama")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Philippines.*$)', "Philippines (the)")
df['Country Name'] = df['Country Name'].replace("Republic of the Congo",'Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Myanmar.*$)', "Myanmar")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Puerto Rico.*$)', "Puerto Rico")
df['Country Name'] = df['Country Name'].replace("Russia",'Russian Federation (the)')
df['Country Name'] = df['Country Name'].replace("Russian Federation",'Russian Federation (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Slovak.*$)', "Slovakia")
df['Country Name'] = df['Country Name'].str.replace(r'\bSudan\b', 'Sudan (the)')
df['Country Name'] = df['Country Name'].str.replace(r'\bSudan (the)\b', 'Sudan (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*South Sudan.*$)', "South Sudan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Syria.*$)', "Syrian Arab Republic (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*São Tomé.*$)', "Sao Tome and Principe")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Taiwan.*$)', "Taiwan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Taipei.*$)', "Taiwan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Tanzania.*$)', "United Republic of Tanzania (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Netherlands.*$)', "Netherlands (the)")
df['Country Name'] = df['Country Name'].replace("UAE",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace("U.A.E",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace("United Arab Emirates",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace('United Kingdom','United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace('UK','United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace("Great Britain",'United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace("United Kingdom of Great Britain and Northern Ireland",'United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace('Vietnam','Viet Nam')
df['Country Name'] = df['Country Name'].replace('United States','United States of America (the)')
df['Country Name'] = df['Country Name'].replace('USA','United States of America (the)')
df['Country Name'] = df['Country Name'].replace('United States of America','United States of America (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Virgin Islands.*$)', "United States Virgin Islands")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vatican.*$)', "Vatican")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Venezuela.*$)', "Venezuela (Bolivarian Republic of)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Yemen.*$)', "Yemen")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Arab world.*$)', "Arab World")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*World.*$)', "World")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kitts and Nevis.*$)', "Saint Kitts and Nevis")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Lucia.*$)', "Saint Lucia")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Martin (French Part).*$)', "Saint Martin (French Part)")
df['Country Name'] = df['Country Name'].replace('Sint Maarten','Saint Martin')
df['Country Name'] = df['Country Name'].replace('St. Martin (French part)','Saint Martin (French Part)')
df['Country Name'] = df['Country Name'].replace('Sint Maarten (Dutch part)','Saint Martin (Dutch Part)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vincent and the Grenadines.*$)', "Saint Vincent and the Grenadines")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Verde.*$)', "Cabo Verde")
df['Country Name'] = df['Country Name'].replace('Congo, Democratic Republic','Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace('Congo, Rep.','Congo (the)')
df['Country Name'] = df['Country Name'].replace('Republic of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].replace('Congo (Rep.)','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Egypt.*$)', "Egypt")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Korea, D.*$)', "Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Tobago.*$)', "Trinidad and Tobago")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Timor-Leste.*$)', "Timor-Leste")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Emirates.*$)', "United Arab Emirates (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Papua.*$)', "Papua New Guinea")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bissau.*$)', "Guinea-Bissau")
df['Country Name'] = df['Country Name'].replace('Eq. Guinea','Equatorial Guinea')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Burma.*$)', "Myanmar")
df['Country Name'] = df['Country Name'].replace('C.A. Republic','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace('Ant.& Barb.','Antigua and Barbuda')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bosnia.*$)', "Bosnia and Herzegovina")
df['Country Name'] = df['Country Name'].replace('Domin. Rep.','Dominican Republic (the)')
df['Country Name'] = df['Country Name'].replace('Dominica (Commonwealth of)','Dominica')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*European Union.*$)', "European Union")
df['Country Name'] = df['Country Name'].replace('R. of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Principe.*$)', "Sao Tome and Principe")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Solomon.*$)', "Solomon Islands")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vincent.*$)', "Saint Vincent and the Grenadines")
df['Country Name'] = df['Country Name'].replace('Curacao','Curaçao')
df['Country Name'] = df['Country Name'].replace('Reunion','Réunion')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kosovo.*$)', "Kosovo (UNSCR 1244)")

pop = df
pop


<ipython-input-19-35a075b1c6b0>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bahamas.*$)', 'Bahamas (the)')
<ipython-input-19-35a075b1c6b0>:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r"(^.*Côte d'Ivoire.*$)", "Côte d'Ivoire")
<ipython-input-19-35a075b1c6b0>:36: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Czech.*$)', 'Czechia')
<ipython-input-19-35a075b1c6b0>:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Dominican Re.*$)', 'Dominican Republic (the)')
<ipython-input-19-35a075b1c6b0>:38: FutureWarning: The default value of regex will change from True to 

1960         1961         1962         1963         1964  \
0        54208.0      55434.0      56234.0      56699.0      57029.0   
1    130836765.0  134159786.0  137614644.0  141202036.0  144920186.0   
2      8996967.0    9169406.0    9351442.0    9543200.0    9744772.0   
3     96396419.0   98407221.0  100506960.0  102691339.0  104953470.0   
4      5454938.0    5531451.0    5608499.0    5679409.0    5734995.0   
..           ...          ...          ...          ...          ...   
261     947000.0     966000.0     994000.0    1022000.0    1050000.0   
262    5315351.0    5393034.0    5473671.0    5556767.0    5641598.0   
263   17099836.0   17524533.0   17965733.0   18423157.0   18896303.0   
264    3070780.0    3164330.0    3260645.0    3360099.0    3463211.0   
265    3776679.0    3905038.0    4039209.0    4178726.0    4322854.0   

            1965         1966         1967         1968         1969  ...  \
0        57357.0      57702.0      58044.0      58377.0      58734.0  ...   
1    148769974.0  152752671.0  156876454.0  161156430.0  165611760.0  ...   
2      9956318.0   10174840.0   10399936.0   10637064.0   10893772.0  ...   
3    107289875.0  109701811.0  112195950.0  114781116.0  117468741.0  ...   
4      5770573.0    5781305.0    5774440.0    5771973.0    5803677.0  ...   
..           ...          ...          ...          ...          ...  ...   
261    1078000.0    1106000.0    1135000.0    1163000.0    1191000.0  ...   
262    5727745.0    5816241.0    5907873.0    6001858.0    6097042.0  ...   
263   19384838.0   19888259.0   20406863.0   20942147.0   21496075.0  ...   
264    3570466.0    3681953.0    3797877.0    3918872.0    4045740.0  ...   
265    4471178.0    4623340.0    4779825.0    4941901.0    5111326.0  ...   

            2015         2016         2017         2018         2019  \
0       104339.0     104865.0     105361.0     105846.0     106310.0   
1    593871847.0  609978946.0  626392880.0  643090131.0  660046272.0   
2     34413603.0   35383028.0   36296111.0   37171922.0   38041757.0   
3    401586651.0  412551299.0  423769930.0  435229381.0  446911598.0   
4     27884380.0   28842482.0   29816769.0   30809787.0   31825299.0   
..           ...          ...          ...          ...          ...   
261    1788196.0    1777557.0    1791003.0    1797085.0    1788878.0   
262   26497881.0   27168210.0   27834811.0   28498683.0   29161922.0   
263   55386369.0   56207649.0   57009751.0   57792520.0   58558267.0   
264   15879370.0   16363449.0   16853608.0   17351714.0   17861034.0   
265   13814642.0   14030338.0   14236599.0   14438812.0   14645473.0   

            2020                 Country Name  Country Code  \
0       106766.0                        Aruba           ABW   
1    677243299.0  Africa Eastern and Southern           AFE   
2     38928341.0                  Afghanistan           AFG   
3    458803476.0   Africa Western and Central           AFW   
4     32866268.0                       Angola           AGO   
..           ...                          ...           ...   
261    1775378.0          Kosovo (UNSCR 1244)           XKX   
262   29825968.0                        Yemen           YEM   
263   59308690.0                 South Africa           ZAF   
264   18383956.0                       Zambia           ZMB   
265   14862927.0                     Zimbabwe           ZWE   

        Indicator Name  Indicator Code  
0    Population, total     SP.POP.TOTL  
1    Population, total     SP.POP.TOTL  
2    Population, total     SP.POP.TOTL  
3    Population, total     SP.POP.TOTL  
4    Population, total     SP.POP.TOTL  
..                 ...             ...  
261  Population, total     SP.POP.TOTL  
262  Population, total     SP.POP.TOTL  
263  Population, total     SP.POP.TOTL  
264  Population, total     SP.POP.TOTL  
265  Population, total     SP.POP.TOTL  

[266 rows x 65 columns]

In [20]:
### Add country's area dataset
### Add population dataset
df = pd.read_csv('../../processed/GDP.csv')

# Normalize country names as much as possible
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bahamas.*$)', 'Bahamas (the)')
df['Country Name'] = df['Country Name'].replace('Bahrain (Kingdom of)','Bahrain')
df['Country Name'] = df['Country Name'].replace('Bolivia','Bolivia (Plurinational State of)')
df['Country Name'] = df['Country Name'].replace('Bolivia, Plurinational State of','Bolivia (Plurinational State of)')
df['Country Name'] = df['Country Name'].replace('Brunei','Brunei Darussalam')
df['Country Name'] = df['Country Name'].replace('Bulgaria (Rep.)','Bulgaria')
df['Country Name'] = df['Country Name'].replace('Central African Republic','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace('Central African Rep.','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace("China (People's Rep.)",'China')
df['Country Name'] = df['Country Name'].replace("Comoros",'Comoros (the)')
df['Country Name'] = df['Country Name'].replace("Congo",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Brazzaville)",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Rep. of the)",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Democratic Republic of the)",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo, Dem. Rep.",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo, The Democratic Republic of the",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("DR Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Democratic Republic of Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Democratic Republic of the Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Dem. Rep. of the Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Cote d'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Côte d’Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cote D'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cote dIvoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].str.replace(r"(^.*Côte d'Ivoire.*$)", "Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cōte d'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Ivory Coast","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Dem. People's Rep. of Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("Democratic People's Republic of Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("Korea, Dem. People's Rep.","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("North Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Czech.*$)', 'Czechia')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Dominican Re.*$)', 'Dominican Republic (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Hong Kong.*$)', 'China, Hong Kong Special Administrative Region')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Hongkong.*$)', 'China, Hong Kong Special Administrative Region')
df['Country Name'] = df['Country Name'].replace("Eswatini (Kingdom of)",'Eswatini')
df['Country Name'] = df['Country Name'].replace("Swaziland",'Eswatini')
df['Country Name'] = df['Country Name'].replace("Faröe Islands",'Faroe Islands')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Gambia.*$)', 'Gambia (the)')
df['Country Name'] = df['Country Name'].replace("Georgia (Country)",'Georgia')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Iran.*$)', 'Iran (Islamic Republic of)')
df['Country Name'] = df['Country Name'].replace("Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Rep. of)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Rep.)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Republic of)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Korea, Rep.*$)', 'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea, South",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("South Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Republic of Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace('Republic of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kyrgyz.*$)', 'Kyrgyzstan')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Lao.*$)', "Lao People's Democratic Republic (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macao.*$)', "China, Macao Special Administrative Region")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macau.*$)', "China, Macao Special Administrative Region")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Micronesia.*$)', "Micronesia (Federated States of)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Moldova.*$)', "Republic of Moldova (the)")
df['Country Name'] = df['Country Name'].replace("Morroco",'Morocco')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Nepal.*$)', "Nepal")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*New Ze.*$)', "New Zealand")
df['Country Name'] = df['Country Name'].replace("Niger",'Niger (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macedonia.*$)', "North Macedonia")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*New Ze.*$)', "New Zealand")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Palestin.*$)', "State of Palestine (the)")
df['Country Name'] = df['Country Name'].replace("West Bank and Gaza",'State of Palestine (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Panama.*$)', "Panama")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Philippines.*$)', "Philippines (the)")
df['Country Name'] = df['Country Name'].replace("Republic of the Congo",'Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Myanmar.*$)', "Myanmar")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Puerto Rico.*$)', "Puerto Rico")
df['Country Name'] = df['Country Name'].replace("Russia",'Russian Federation (the)')
df['Country Name'] = df['Country Name'].replace("Russian Federation",'Russian Federation (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Slovak.*$)', "Slovakia")
df['Country Name'] = df['Country Name'].str.replace(r'\bSudan\b', 'Sudan (the)')
df['Country Name'] = df['Country Name'].str.replace(r'\bSudan (the)\b', 'Sudan (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*South Sudan.*$)', "South Sudan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Syria.*$)', "Syrian Arab Republic (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*São Tomé.*$)', "Sao Tome and Principe")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Taiwan.*$)', "Taiwan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Taipei.*$)', "Taiwan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Tanzania.*$)', "United Republic of Tanzania (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Netherlands.*$)', "Netherlands (the)")
df['Country Name'] = df['Country Name'].replace("UAE",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace("U.A.E",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace("United Arab Emirates",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace('United Kingdom','United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace('UK','United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace("Great Britain",'United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace("United Kingdom of Great Britain and Northern Ireland",'United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace('Vietnam','Viet Nam')
df['Country Name'] = df['Country Name'].replace('United States','United States of America (the)')
df['Country Name'] = df['Country Name'].replace('USA','United States of America (the)')
df['Country Name'] = df['Country Name'].replace('United States of America','United States of America (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Virgin Islands.*$)', "United States Virgin Islands")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vatican.*$)', "Vatican")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Venezuela.*$)', "Venezuela (Bolivarian Republic of)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Yemen.*$)', "Yemen")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Arab world.*$)', "Arab World")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*World.*$)', "World")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kitts and Nevis.*$)', "Saint Kitts and Nevis")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Lucia.*$)', "Saint Lucia")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Martin (French Part).*$)', "Saint Martin (French Part)")
df['Country Name'] = df['Country Name'].replace('Sint Maarten','Saint Martin')
df['Country Name'] = df['Country Name'].replace('St. Martin (French part)','Saint Martin (French Part)')
df['Country Name'] = df['Country Name'].replace('Sint Maarten (Dutch part)','Saint Martin (Dutch Part)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vincent and the Grenadines.*$)', "Saint Vincent and the Grenadines")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Verde.*$)', "Cabo Verde")
df['Country Name'] = df['Country Name'].replace('Congo, Democratic Republic','Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace('Congo, Rep.','Congo (the)')
df['Country Name'] = df['Country Name'].replace('Republic of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].replace('Congo (Rep.)','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Egypt.*$)', "Egypt")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Korea, D.*$)', "Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Tobago.*$)', "Trinidad and Tobago")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Timor-Leste.*$)', "Timor-Leste")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Emirates.*$)', "United Arab Emirates (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Papua.*$)', "Papua New Guinea")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bissau.*$)', "Guinea-Bissau")
df['Country Name'] = df['Country Name'].replace('Eq. Guinea','Equatorial Guinea')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Burma.*$)', "Myanmar")
df['Country Name'] = df['Country Name'].replace('C.A. Republic','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace('Ant.& Barb.','Antigua and Barbuda')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bosnia.*$)', "Bosnia and Herzegovina")
df['Country Name'] = df['Country Name'].replace('Domin. Rep.','Dominican Republic (the)')
df['Country Name'] = df['Country Name'].replace('Dominica (Commonwealth of)','Dominica')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*European Union.*$)', "European Union")
df['Country Name'] = df['Country Name'].replace('R. of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Principe.*$)', "Sao Tome and Principe")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Solomon.*$)', "Solomon Islands")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vincent.*$)', "Saint Vincent and the Grenadines")
df['Country Name'] = df['Country Name'].replace('Curacao','Curaçao')
df['Country Name'] = df['Country Name'].replace('Reunion','Réunion')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kosovo.*$)', "Kosovo (UNSCR 1244)")

gdp = df
gdp


<ipython-input-20-25eba6b4856e>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bahamas.*$)', 'Bahamas (the)')
<ipython-input-20-25eba6b4856e>:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r"(^.*Côte d'Ivoire.*$)", "Côte d'Ivoire")
<ipython-input-20-25eba6b4856e>:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Czech.*$)', 'Czechia')
<ipython-input-20-25eba6b4856e>:38: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Dominican Re.*$)', 'Dominican Republic (the)')
<ipython-input-20-25eba6b4856e>:39: FutureWarning: The default value of regex will change from True to 

1960          1961          1962          1963          1964  \
0             NaN           NaN           NaN           NaN           NaN   
1    1.934248e+10  1.975349e+10  2.152662e+10  2.577236e+10  2.356323e+10   
2    5.377778e+08  5.488889e+08  5.466667e+08  7.511112e+08  8.000000e+08   
3    1.040732e+10  1.113130e+10  1.194684e+10  1.268022e+10  1.384262e+10   
4             NaN           NaN           NaN           NaN           NaN   
..            ...           ...           ...           ...           ...   
261           NaN           NaN           NaN           NaN           NaN   
262           NaN           NaN           NaN           NaN           NaN   
263  7.575397e+09  7.972997e+09  8.497997e+09  9.423396e+09  1.037400e+10   
264  7.130000e+08  6.962857e+08  6.931429e+08  7.187143e+08  8.394286e+08   
265  1.052990e+09  1.096647e+09  1.117602e+09  1.159512e+09  1.217138e+09   

             1965          1966          1967          1968          1969  \
0             NaN           NaN           NaN           NaN           NaN   
1    2.685135e+10  2.919650e+10  3.021907e+10  3.292707e+10  3.780176e+10   
2    1.006667e+09  1.400000e+09  1.673333e+09  1.373333e+09  1.408889e+09   
3    1.486682e+10  1.583747e+10  1.443065e+10  1.488470e+10  1.688703e+10   
4             NaN           NaN           NaN           NaN           NaN   
..            ...           ...           ...           ...           ...   
261           NaN           NaN           NaN           NaN           NaN   
262           NaN           NaN           NaN           NaN           NaN   
263  1.133440e+10  1.235500e+10  1.377739e+10  1.489459e+10  1.678039e+10   
264  1.082857e+09  1.264286e+09  1.368000e+09  1.605857e+09  1.965714e+09   
265  1.311436e+09  1.281750e+09  1.397002e+09  1.479600e+09  1.747999e+09   

     ...          2015          2016          2017          2018  \
0    ...  2.962905e+09  2.983637e+09  3.092430e+09  3.202189e+09   
1    ...  8.954400e+11  8.569920e+11  9.647910e+11  9.866110e+11   
2    ...  1.990711e+10  1.801775e+10  1.886995e+10  1.835388e+10   
3    ...  7.574920e+11  6.874850e+11  6.809890e+11  7.381310e+11   
4    ...  1.161940e+11  1.011240e+11  1.221240e+11  1.013530e+11   
..   ...           ...           ...           ...           ...   
261  ...  6.442916e+09  6.719172e+09  7.245707e+09  7.942962e+09   
262  ...  4.244510e+10  3.093383e+10  2.673614e+10  2.348627e+10   
263  ...  3.176210e+11  2.963570e+11  3.495540e+11  3.682890e+11   
264  ...  2.125122e+10  2.095841e+10  2.587360e+10  2.631164e+10   
265  ...  1.996312e+10  2.054868e+10  1.901533e+10  1.952362e+10   

             2019          2020                 Country Name  Country Code  \
0             NaN           NaN                        Aruba           ABW   
1    9.803720e+11  9.008290e+11  Africa Eastern and Southern           AFE   
2    1.929110e+10  1.980707e+10                  Afghanistan           AFG   
3    7.920790e+11  7.865850e+11   Africa Western and Central           AFW   
4    8.941719e+10  6.230691e+10                       Angola           AGO   
..            ...           ...                          ...           ...   
261  7.953156e+09  7.611402e+09          Kosovo (UNSCR 1244)           XKX   
262           NaN           NaN                        Yemen           YEM   
263  3.514320e+11  3.019240e+11                 South Africa           ZAF   
264  2.330869e+10  1.932005e+10                       Zambia           ZMB   
265  1.693243e+10  1.676851e+10                     Zimbabwe           ZWE   

        Indicator Name  Indicator Code  
0    GDP (current US$)  NY.GDP.MKTP.CD  
1    GDP (current US$)  NY.GDP.MKTP.CD  
2    GDP (current US$)  NY.GDP.MKTP.CD  
3    GDP (current US$)  NY.GDP.MKTP.CD  
4    GDP (current US$)  NY.GDP.MKTP.CD  
..                 ...             ...  
261  GDP (current US$)  NY.GDP.MKTP.CD  
262  GDP (current US$)  NY.GDP.MKTP.CD  
263  GDP (current US$)  NY.GDP.MKTP.CD  

### 1. 'UNCTAD Business-to-Consumer (B2C) E-commerce Index'

In [21]:
indicators[0]

'UNCTAD Business-to-Consumer (B2C) E-commerce Index'

In [22]:
# load data
indicator = indicators[0]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

UNCTAD Business-to-Consumer (B2C) E-commerce Index
b2c_ecommerse_idx


In [23]:
df.head()

2020 Rannk         Economy  \
0           1     Switzerland   
1           2     Netherlands   
2           3         Denmark   
3           4       Singapore   
4           5  United Kingdom   

   Share of individuals using the Internet (2019 or latest)  \
0                                                 97          
1                                                 96          
2                                                 97          
3                                                 89          
4                                                 96          

   Share of individuals with an account (15+, 2017)  \
0                                                98   
1                                               100   
2                                               100   
3                                                98   
4                                                96   

   Secure Internet servers (normalize d, 2019)  \
0                                           92   
1                                           94   
2                                          100   
3                                           94   
4                                           84   

   UPU postal reliability score (2019 or latest)  2020 Index value  \
0                                             97              95.9   
1                                             93              95.8   
2                                             81              94.5   
3                                             97              94.4   
4                                             98              93.6   

   Index value change (2019-20data)  Rank 2019  
0                              -0.1          2  
1                              -0.7          1  
2                               0.2          6  
3                              -0.4          3  
4                              -0.8          4

In [24]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Technology Adoption


In [25]:
# create standard columns
df.rename(columns={'Economy':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['2020 Index value'] 
df['Sub-Pillar'] = subpillar
df['Year'] = 2020

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [26]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

Country Name  Year  \
0               Switzerland  2020   
1               Netherlands  2020   
2                   Denmark  2020   
3                 Singapore  2020   
4            United Kingdom  2020   
..                      ...   ...   
147  Dem. Rep. of the Congo  2020   
148                 Comoros  2020   
149                 Burundi  2020   
150                    Chad  2020   
151                   Niger  2020   

                                             Indicator  data_col  \
0    UNCTAD Business-to-Consumer (B2C) E-commerce I...      95.9   
1    UNCTAD Business-to-Consumer (B2C) E-commerce I...      95.8   
2    UNCTAD Business-to-Consumer (B2C) E-commerce I...      94.5   
3    UNCTAD Business-to-Consumer (B2C) E-commerce I...      94.4   
4    UNCTAD Business-to-Consumer (B2C) E-commerce I...      93.6   
..                                                 ...       ...   
147  UNCTAD Business-to-Consumer (B2C) E-commerce I...      12.8   
148  UNCTAD Business-to-Consumer (B2C) E-commerce I...      12.0   
149  UNCTAD Business-to-Consumer (B2C) E-commerce I...       8.3   
150  UNCTAD Business-to-Consumer (B2C) E-commerce I...       7.1   
151  UNCTAD Business-to-Consumer (B2C) E-commerce I...       5.6   

     new_rank_score  higher_is_better           Sub-Pillar  
0           5.78541              True  Technology Adoption  
1           5.78042              True  Technology Adoption  
2           5.71555              True  Technology Adoption  
3           5.71056              True  Technology Adoption  
4           5.67064              True  Technology Adoption  
..              ...               ...                  ...  
147         1.63872              True  Technology Adoption  
148         1.59880              True  Technology Adoption  
149         1.41417              True  Technology Adoption  
150         1.35429              True  Technology Adoption  
151         1.27944              True  Technology Adoption  

[152 rows x 7 columns]

In [27]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator),index = False)

## 2. Networking Services (Spend, IT Forecast Data)


In [28]:
indicator = indicators[1]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Networking Services (Spend, IT Forecast Data)
network_readiness_index


In [29]:
df.head(16)

Rank          Country  Score  Income Group           Region
0      1          Sweden   82.75  High-income           Europe 
1      2         Denmark   82.19  High-income           Europe 
2      3       Singapore   81.39  High-income   Asia & Pacific 
3      4     Netherlands   81.37  High-income           Europe 
4      5     Switzerland   80.41  High-income           Europe 
5      6         Finland   80.16  High-income           Europe 
6      7          Norway   79.39  High-income           Europe 
7      8   United States   78.91  High-income     The Americas 
8      9         Germany   77.48  High-income           Europe 
9     10  United Kingdom   76.27  High-income            Europe
10    11      Luxembourg   75.27  High-income           Europe 
11    12       Australia   75.09  High-income   Asia & Pacific 
12    13          Canada   74.92  High-income     The Americas 
13    14     Korea, Rep.   74.60  High-income   Asia & Pacific 
14    15           Japan   73.54  High-income   Asia & Pacific 
15    16     New Zealand   73.27  High-income   Asia & Pacific

In [30]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Technology Adoption


In [31]:
# convert rank into 1-6 - in order to get most recent data 
df['data_col'] = df['Score'] 


# going to assume index is between 1-100 but not 100% sure
min_rank = 1 #df['data_col'].min()
max_rank = 100 #df['data_col'].max()

# transform 1-100 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

# need to invert score since higher rank is not better 
# df['new_rank_score'] = (6-df['new_rank_score'])+1

In [32]:
# prepare output
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Indicator'] = indicator
df['higher_is_better'] = True
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

In [33]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

Country Name  Year                                      Indicator  \
0              Sweden   2020  Networking Services (Spend, IT Forecast Data)   
1             Denmark   2020  Networking Services (Spend, IT Forecast Data)   
2           Singapore   2020  Networking Services (Spend, IT Forecast Data)   
3         Netherlands   2020  Networking Services (Spend, IT Forecast Data)   
4         Switzerland   2020  Networking Services (Spend, IT Forecast Data)   
..                 ...   ...                                            ...   
129           Burundi   2020  Networking Services (Spend, IT Forecast Data)   
130            Angola   2020  Networking Services (Spend, IT Forecast Data)   
131             Yemen   2020  Networking Services (Spend, IT Forecast Data)   
132  Congo, Dem. Rep.   2020  Networking Services (Spend, IT Forecast Data)   
133              Chad   2020  Networking Services (Spend, IT Forecast Data)   

     data_col  new_rank_score  higher_is_better           Sub-Pillar  
0       82.75        5.129225              True  Technology Adoption  
1       82.19        5.101281              True  Technology Adoption  
2       81.39        5.061361              True  Technology Adoption  
3       81.37        5.060363              True  Technology Adoption  
4       80.41        5.012459              True  Technology Adoption  
..        ...             ...               ...                  ...  
129     22.62        2.128738              True  Technology Adoption  
130     20.96        2.045904              True  Technology Adoption  
131     18.00        1.898200              True  Technology Adoption  
132     16.60        1.828340              True  Technology Adoption  
133     14.80        1.738520              True  Technology Adoption  

[134 rows x 7 columns]

In [34]:
# output scores
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator),index = False)

### 3. Cloud Services (Spend, IT Forecast Data)


In [35]:
indicator = indicators[2]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Cloud Services (Spend, IT Forecast Data)
cloud_services


In [36]:
# remove nulls
df = df.dropna()
df

Cloud computing policy environment by category - country ranking 2018  \
2                                             Germany                      
3                                               Japan                      
4                                       United States                      
5                                      United Kingdom                      
6                                           Australia                      
7                                           Singapore                      
8                                              Canada                      
9                                              France                      
10                                              Italy                      
11                                              Spain                      
12                                             Poland                      
13                                              Korea                      
14                                             Mexico                      
15                                           Malaysia                      
16                                       South Africa                      
17                                             Turkey                      
18                                          Argentina                      
19                                             Brazil                      
20                                           Thailand                      
21                                              India                      
22                                             Russia                      
23                                              China                      
24                                          Indonesia                      
25                                            Vietnam                      

   Unnamed: 1  
2       18.20  
3       20.30  
4          18  
5       19.80  
6       16.10  
7       20.70  
8          17  
9       17.30  
10         15  
11      16.60  
12      14.70  
13      19.90  
14       9.10  
15      13.50  
16      10.30  
17      11.10  
18      10.80  
19      11.50  
20      11.10  
21       6.90  
22      15.70  
23      12.70  
24       8.20  
25       8.60

In [37]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Technology Adoption


In [38]:
# prepare standard columns
df['data_col'] = df['Unnamed: 1'].astype(float)
df['Country Name'] = df.iloc[:,0]
df['Indicator'] = indicator
df['higher_is_better'] = True
df['Year'] = 2018
df['Sub-Pillar'] = subpillar

In [39]:
min_rank = 1
max_rank = df['Country Name'].nunique()

In [40]:
# transform 1-24 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

# need to invert score since higher rank is not better 
# df['new_rank_score'] = (6-df['new_rank_score'])+1

In [41]:
# prepare output
df.sort_values(by='new_rank_score', ascending=False)

# prepare output
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]

# output scores
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

In [42]:
df.head(15)

Country Name  Year                                 Indicator  data_col  \
2          Germany  2018  Cloud Services (Spend, IT Forecast Data)      18.2   
3            Japan  2018  Cloud Services (Spend, IT Forecast Data)      20.3   
4    United States  2018  Cloud Services (Spend, IT Forecast Data)      18.0   
5   United Kingdom  2018  Cloud Services (Spend, IT Forecast Data)      19.8   
6        Australia  2018  Cloud Services (Spend, IT Forecast Data)      16.1   
7        Singapore  2018  Cloud Services (Spend, IT Forecast Data)      20.7   
8           Canada  2018  Cloud Services (Spend, IT Forecast Data)      17.0   
9           France  2018  Cloud Services (Spend, IT Forecast Data)      17.3   
10           Italy  2018  Cloud Services (Spend, IT Forecast Data)      15.0   
11           Spain  2018  Cloud Services (Spend, IT Forecast Data)      16.6   
12          Poland  2018  Cloud Services (Spend, IT Forecast Data)      14.7   
13           Korea  2018  Cloud Services (Spend, IT Forecast Data)      19.9   
14          Mexico  2018  Cloud Services (Spend, IT Forecast Data)       9.1   
15        Malaysia  2018  Cloud Services (Spend, IT Forecast Data)      13.5   
16    South Africa  2018  Cloud Services (Spend, IT Forecast Data)      10.3   

    new_rank_score  higher_is_better           Sub-Pillar  
2          1.90818              True  Technology Adoption  
3          2.01297              True  Technology Adoption  
4          1.89820              True  Technology Adoption  
5          1.98802              True  Technology Adoption  
6          1.80339              True  Technology Adoption  
7          2.03293              True  Technology Adoption  
8          1.84830              True  Technology Adoption  
9          1.86327              True  Technology Adoption  
10         1.74850              True  Technology Adoption  
11         1.82834              True  Technology Adoption  
12         1.73353              True  Technology Adoption  
13         1.99301              True  Technology Adoption  
14         1.45409              True  Technology Adoption  
15         1.67365              True  Technology Adoption  
16         1.51397              True  Technology Adoption

## 4. ICT task-intensive jobs as a percentage of total employment

In [43]:
indicator = indicators[3]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

ICT task-intensive jobs as a percentage of total employment
ICT_proportion


In [44]:
df.head()

Indicator  Country       Industry  \
0  ICT specialists within and outside information...  Austria  Total economy   
1  ICT specialists within and outside information...  Austria  Total economy   
2  ICT specialists within and outside information...  Austria  Total economy   
3  ICT specialists within and outside information...  Austria  Total economy   
4  ICT specialists within and outside information...  Austria  Total economy   

          Information and communication technologies    Sex        Measure  \
0  Specialist (ISCO-08: 133+215+251+252+351+352+742)  Total  Share of jobs   
1  Specialist (ISCO-08: 133+215+251+252+351+352+742)  Total  Share of jobs   
2  Specialist (ISCO-08: 133+215+251+252+351+352+742)  Total  Share of jobs   
3  Specialist (ISCO-08: 133+215+251+252+351+352+742)  Total  Share of jobs   
4  Specialist (ISCO-08: 133+215+251+252+351+352+742)  Total  Share of jobs   

   Time     Value  Flags  
0  2011  3.176400    NaN  
1  2012  3.225967    NaN  
2  2013  3.346251    NaN  
3  2014  3.319100    NaN  
4  2015  3.729340    NaN

In [45]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Technology Adoption


In [46]:
df[(df['Time']==2018)&(df['Information and communication technologies']=='ICT-intensive')].sort_values(by='Value', ascending=False)

Empty DataFrame
Columns: [Indicator, Country, Industry, Information and communication technologies, Sex, Measure, Time, Value, Flags]
Index: []

In [47]:
# bnames

In [48]:
df['Information and communication technologies'].unique()

array(['Specialist (ISCO-08: 133+215+251+252+351+352+742)',
       'Other ICT-intensive (ISCO-08: 121+122,134+,211+,216+,231+,241+,242+243)',
       'Non-ICT (rest of ISCO-08 occupations)', 'ICT-intensive', 'Total'],
      dtype=object)

In [49]:
df.Sex.unique()

array(['Total'], dtype=object)

In [50]:
# convert to correct types
df['Value'] = df['Value'].astype(float)

In [51]:
df['Value'].describe()

count    985.000000
mean      42.353406
std       42.614469
min        0.890157
25%        5.526795
50%       12.887070
75%       91.318100
max      100.000000
Name: Value, dtype: float64

In [52]:
# filter on relevant years
df = df[(df['Time']==2017)&(df['Information and communication technologies']=='ICT-intensive')]

# convert rank into 1-6 - in order to get most recent data 
df['data_col'] = df['Value'] 

# going to assume index is between 1-100 but not 100% sure
min_rank = 0 #df['data_col'].min()
max_rank = 100 #df['data_col'].max()

# transform 1-147 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

# need to invert score since higher rank is not better 
# df['new_rank_score'] = (6-df['new_rank_score'])+1

# df_rank[['Country ISO3', 'Country Name','Indicator','data_col','new_rank_score']].head()

df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Indicator'] = indicator
df['higher_is_better'] = True
df['Year'] = df['Time']
df['Sub-Pillar'] = subpillar

df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better']].head(15)


# output scores to csv
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

Country Name  Year                                          Indicator  \
27           Austria  2017  ICT task-intensive jobs as a percentage of tot...   
62           Belgium  2017  ICT task-intensive jobs as a percentage of tot...   
97    Czech Republic  2017  ICT task-intensive jobs as a percentage of tot...   
157          Estonia  2017  ICT task-intensive jobs as a percentage of tot...   
217          Finland  2017  ICT task-intensive jobs as a percentage of tot...   
252           France  2017  ICT task-intensive jobs as a percentage of tot...   
287          Germany  2017  ICT task-intensive jobs as a percentage of tot...   
322           Greece  2017  ICT task-intensive jobs as a percentage of tot...   
357          Hungary  2017  ICT task-intensive jobs as a percentage of tot...   
392          Iceland  2017  ICT task-intensive jobs as a percentage of tot...   
452            Italy  2017  ICT task-intensive jobs as a percentage of tot...   
507           Latvia  2017  ICT task-intensive jobs as a percentage of tot...   
542        Lithuania  2017  ICT task-intensive jobs as a percentage of tot...   
577       Luxembourg  2017  ICT task-intensive jobs as a percentage of tot...   
612      Netherlands  2017  ICT task-intensive jobs as a percentage of tot...   
647           Norway  2017  ICT task-intensive jobs as a percentage of tot...   
682           Poland  2017  ICT task-intensive jobs as a percentage of tot...   
742  Slovak Republic  2017  ICT task-intensive jobs as a percentage of tot...   
777         Slovenia  2017  ICT task-intensive jobs as a percentage of tot...   
812            Spain  2017  ICT task-intensive jobs as a percentage of tot...   
847           Sweden  2017  ICT task-intensive jobs as a percentage of tot...   
882      Switzerland  2017  ICT task-intensive jobs as a percentage of tot...   
942   United Kingdom  2017  ICT task-intensive jobs as a percentage of tot...   
977    United States  2017  ICT task-intensive jobs as a percentage of tot...   

      data_col  new_rank_score  higher_is_better           Sub-Pillar  
27   10.884160        1.543120              True  Technology Adoption  
62   14.448260        1.720968              True  Technology Adoption  
97    9.219953        1.460076              True  Technology Adoption  
157  15.761420        1.786495              True  Technology Adoption  
217  15.220480        1.759502              True  Technology Adoption  
252  12.008350        1.599217              True  Technology Adoption  
287  10.441900        1.521051              True  Technology Adoption  
322   6.675247        1.333095              True  Technology Adoption  
357   8.369766        1.417651              True  Technology Adoption  
392  14.276870        1.712416              True  Technology Adoption  
452   7.254931        1.362021              True  Technology Adoption  
507  12.482960        1.622900              True  Technology Adoption  
542  16.276940        1.812219              True  Technology Adoption  
577  21.685190        2.082091              True  Technology Adoption  
612  15.733610        1.785107              True  Technology Adoption  
647  14.720470        1.734551              True  Technology Adoption  
682  11.125690        1.555172              True  Technology Adoption  
742   6.841666        1.341399              True  Technology Adoption  
777  13.193620        1.658362              True  Technology Adoption  
812   8.421205        1.420218              True  Technology Adoption  
847  16.588810        1.827782              True  Technology Adoption  
882  15.372650        1.767095              True  Technology Adoption  
942  17.392000        1.867861              True  Technology Adoption  
977  17.822500        1.889343              True  Technology Adoption

In [53]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator),index = False)

## 5. Share of business with internet

In [54]:
indicator = indicators[4]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Share of business with internet
business_internet


In [55]:
df= df.replace('..',np.nan)

In [56]:
df.head(15)

2005   2006   2007   2008   2009   2010   2011   2012   2013   2014  \
0   52.47   54.8  60.17  61.52  67.16  67.81  69.33  76.25  75.58  75.77   
1   72.22  78.81  80.06  79.84  79.85  80.19  82.87  82.01   85.7  86.35   
2     NaN    NaN    NaN    NaN  77.37  78.47   76.6  76.01  78.26  79.15   
3    64.8   67.5   69.7    NaN    NaN    NaN    NaN   79.8   77.5    NaN   
4     NaN    NaN    NaN  47.87  51.74  54.03  59.41  63.27  66.55     67   
5     NaN  70.08  71.12  73.99  72.66  73.63  77.44  79.67  79.86  82.63   
6     NaN    NaN    NaN    NaN  87.61  87.83  88.68   89.3  91.78   91.4   
7   52.65  57.86  61.87  65.73  67.53  70.04  72.63  74.97  75.74  77.56   
8     NaN    NaN    NaN    NaN  84.62  87.32  92.56   91.3  93.64   95.1   
9     NaN    NaN    NaN    NaN  54.05  57.71  60.05  64.48   65.3  63.59   
10  71.25  73.87  76.88  75.39  78.94  81.43  81.26  82.03  84.41  85.77   
11   55.3  50.39  57.26  55.44  61.91  57.92  63.95  63.51  60.59  61.85   
12    NaN    NaN  45.25  47.47  50.76  56.56  59.79  61.77  61.23  61.26   
13    NaN    NaN    NaN    NaN    NaN  77.17    NaN  84.48  82.73   84.7   
14    NaN    NaN    NaN    NaN  62.16  68.05  70.12  73.52  74.71   73.2   

     2015   2016   2017   2018   2019   2020         Country  
0   76.73  75.62  77.37  79.38  80.37    NaN       Australia  
1   87.46  88.11  85.55  87.92  89.45  90.42         Austria  
2   81.04     81   82.6  84.03  86.72  86.62         Belgium  
3     NaN    NaN   78.5    NaN   81.8    NaN          Canada  
4   66.47  67.43  67.17  67.81    NaN    NaN        Colombia  
5   82.57  82.15   82.9  82.79  83.31  83.32  Czech Republic  
6   91.95  93.34  95.09  95.58  93.92  92.77         Denmark  
7   79.73  77.93  78.09  78.36  81.18  79.79         Estonia  
8    95.2  95.33  96.28  95.64    NaN  95.92         Finland  
9   66.82   68.5  66.53  69.41  71.54  70.35          France  
10  87.12  88.93  87.32  87.39  88.21  88.35         Germany  
11  60.95  65.89  64.77   64.8   59.3   60.1          Greece  
12  64.46  67.77  68.68  65.72  62.47  63.23         Hungary  
13    NaN    NaN  81.53  82.57    NaN    NaN         Iceland  
14  75.25  76.07  74.19  78.85  78.68  75.27         Ireland

In [57]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Technology Adoption


In [58]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['2020'].astype(float)
df['Country Name'] = df['Country']
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [59]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.head(15)

Country Name  Year                        Indicator  data_col  \
0        Australia  2020  Share of business with internet       NaN   
1          Austria  2020  Share of business with internet     90.42   
2          Belgium  2020  Share of business with internet     86.62   
3           Canada  2020  Share of business with internet       NaN   
4         Colombia  2020  Share of business with internet       NaN   
5   Czech Republic  2020  Share of business with internet     83.32   
6          Denmark  2020  Share of business with internet     92.77   
7          Estonia  2020  Share of business with internet     79.79   
8          Finland  2020  Share of business with internet     95.92   
9           France  2020  Share of business with internet     70.35   
10         Germany  2020  Share of business with internet     88.35   
11          Greece  2020  Share of business with internet     60.10   
12         Hungary  2020  Share of business with internet     63.23   
13         Iceland  2020  Share of business with internet       NaN   
14         Ireland  2020  Share of business with internet     75.27   

    new_rank_score  higher_is_better           Sub-Pillar  
0              NaN              True  Technology Adoption  
1         5.511958              True  Technology Adoption  
2         5.322338              True  Technology Adoption  
3              NaN              True  Technology Adoption  
4              NaN              True  Technology Adoption  
5         5.157668              True  Technology Adoption  
6         5.629223              True  Technology Adoption  
7         4.981521              True  Technology Adoption  
8         5.786408              True  Technology Adoption  
9         4.510465              True  Technology Adoption  
10        5.408665              True  Technology Adoption  
11        3.998990              True  Technology Adoption  
12        4.155177              True  Technology Adoption  
13             NaN              True  Technology Adoption  
14        4.755973              True  Technology Adoption

In [60]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

## 6. Share of businesses with broadband

In [61]:
indicator = indicators[5]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Share of businesses with broadband
business_broadband


In [62]:
df.head(15)

2008   2009   2010          Country
0   76.94  76.01  82.06          Austria
1   79.33  77.31  86.52   Czech Republic
2   87.53  86.08  88.08          Estonia
3     NaN  91.68  93.31           France
4   83.46  87.90  89.34          Germany
5   70.37  74.19  79.61          Hungary
6     NaN    NaN  95.43          Iceland
7     NaN  76.11  86.84          Ireland
8     NaN  82.92  84.12            Italy
9     NaN  88.03  87.91       Luxembourg
10    NaN  86.38  90.93      Netherlands
11  84.36  88.28  86.92           Norway
12  59.12  58.47  69.34           Poland
13    NaN  81.70  84.75         Portugal
14  73.72  78.37  78.32  Slovak Republic

In [63]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Technology Adoption


In [64]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['2010']
df['Country Name'] = df['Country']
df['Year'] = 2010
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [65]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.head(15)

Country Name  Year                           Indicator  data_col  \
0           Austria  2010  Share of businesses with broadband     82.06   
1    Czech Republic  2010  Share of businesses with broadband     86.52   
2           Estonia  2010  Share of businesses with broadband     88.08   
3            France  2010  Share of businesses with broadband     93.31   
4           Germany  2010  Share of businesses with broadband     89.34   
5           Hungary  2010  Share of businesses with broadband     79.61   
6           Iceland  2010  Share of businesses with broadband     95.43   
7           Ireland  2010  Share of businesses with broadband     86.84   
8             Italy  2010  Share of businesses with broadband     84.12   
9        Luxembourg  2010  Share of businesses with broadband     87.91   
10      Netherlands  2010  Share of businesses with broadband     90.93   
11           Norway  2010  Share of businesses with broadband     86.92   
12           Poland  2010  Share of businesses with broadband     69.34   
13         Portugal  2010  Share of businesses with broadband     84.75   
14  Slovak Republic  2010  Share of businesses with broadband     78.32   

    new_rank_score  higher_is_better           Sub-Pillar  
0         5.094794              True  Technology Adoption  
1         5.317348              True  Technology Adoption  
2         5.395192              True  Technology Adoption  
3         5.656169              True  Technology Adoption  
4         5.458066              True  Technology Adoption  
5         4.972539              True  Technology Adoption  
6         5.761957              True  Technology Adoption  
7         5.333316              True  Technology Adoption  
8         5.197588              True  Technology Adoption  
9         5.386709              True  Technology Adoption  
10        5.537407              True  Technology Adoption  
11        5.337308              True  Technology Adoption  
12        4.460066              True  Technology Adoption  
13        5.229025              True  Technology Adoption  
14        4.908168              True  Technology Adoption

In [66]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

## 7. Share of businesses with online presence

In [67]:
indicator = indicators[6]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))
df.head(15)

Share of businesses with online presence
share_of_businesses_online_presence


RANK COUNTRY/ECONOMY  VALUE   SCORE  Year
0    1.0         Finland  95.64  100.00  2018
1    2.0         Denmark  93.92   98.01  2018
2    3.0           Japan  92.40   96.24  2018
3    4.0     Netherlands  91.89   95.65  2018
4    5.0     Switzerland  91.74   95.48  2018
5    6.0          Sweden  89.65   93.05  2018
6    7.0         Austria  89.45   92.82  2018
7    8.0         Germany  88.21   91.38  2018
8    9.0         Belgium  86.72   89.65  2018
9   10.0  United Kingdom  83.88   86.35  2018
10  11.0      Luxembourg  83.67   86.11  2018
11  12.0  Czech Republic  83.31   85.69  2018
12  13.0           Malta  83.30   85.68  2018
13  14.0        Slovenia  82.76   85.05  2018
14  15.0         Iceland  82.57   84.83  2018

In [68]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Technology Adoption


In [69]:
# create standard columns
df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['VALUE']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [70]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.head(15)

Country Name  Year                                 Indicator  data_col  \
0          Finland  2018  Share of businesses with online presence     95.64   
1          Denmark  2018  Share of businesses with online presence     93.92   
2            Japan  2018  Share of businesses with online presence     92.40   
3      Netherlands  2018  Share of businesses with online presence     91.89   
4      Switzerland  2018  Share of businesses with online presence     91.74   
5           Sweden  2018  Share of businesses with online presence     89.65   
6          Austria  2018  Share of businesses with online presence     89.45   
7          Germany  2018  Share of businesses with online presence     88.21   
8          Belgium  2018  Share of businesses with online presence     86.72   
9   United Kingdom  2018  Share of businesses with online presence     83.88   
10      Luxembourg  2018  Share of businesses with online presence     83.67   
11  Czech Republic  2018  Share of businesses with online presence     83.31   
12           Malta  2018  Share of businesses with online presence     83.30   
13        Slovenia  2018  Share of businesses with online presence     82.76   
14         Iceland  2018  Share of businesses with online presence     82.57   

    new_rank_score  higher_is_better           Sub-Pillar  
0         5.772436              True  Technology Adoption  
1         5.686608              True  Technology Adoption  
2         5.610760              True  Technology Adoption  
3         5.585311              True  Technology Adoption  
4         5.577826              True  Technology Adoption  
5         5.473535              True  Technology Adoption  
6         5.463555              True  Technology Adoption  
7         5.401679              True  Technology Adoption  
8         5.327328              True  Technology Adoption  
9         5.185612              True  Technology Adoption  
10        5.175133              True  Technology Adoption  
11        5.157169              True  Technology Adoption  
12        5.156670              True  Technology Adoption  
13        5.129724              True  Technology Adoption  
14        5.120243              True  Technology Adoption

In [71]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

## 8. Size of gig economy (% of GDP)

In [72]:
indicator = indicators[7]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Size of gig economy (% of GDP)
prevalance_gig_economy


In [73]:
df.head(15)

RANK       COUNTRY/ECONOMY  VALUE   SCORE
0    1.0         United States   5.40  100.00
1    2.0           Netherlands   5.22   94.63
2    3.0        United Kingdom   5.19   93.80
3    4.0          Saudi Arabia   5.08   90.33
4    5.0              Malaysia   5.07   90.19
5    6.0                 Egypt   5.05   89.46
6    7.0                Israel   5.02   88.42
7    8.0                Canada   4.94   86.07
8    9.0             Singapore   4.92   85.52
9   10.0  United Arab Emirates   4.87   83.82
10  11.0             Australia   4.81   82.16
11  12.0               Germany   4.81   82.07
12  13.0                 Qatar   4.57   74.83
13  14.0           New Zealand   4.54   73.86
14  15.0            Azerbaijan   4.53   73.58

In [74]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Technology Adoption


In [75]:
# create standard columns
df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['SCORE']
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [76]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.head(15)

Country Name  Year                       Indicator  data_col  \
0          United States  2019  Size of gig economy (% of GDP)    100.00   
1            Netherlands  2019  Size of gig economy (% of GDP)     94.63   
2         United Kingdom  2019  Size of gig economy (% of GDP)     93.80   
3           Saudi Arabia  2019  Size of gig economy (% of GDP)     90.33   
4               Malaysia  2019  Size of gig economy (% of GDP)     90.19   
5                  Egypt  2019  Size of gig economy (% of GDP)     89.46   
6                 Israel  2019  Size of gig economy (% of GDP)     88.42   
7                 Canada  2019  Size of gig economy (% of GDP)     86.07   
8              Singapore  2019  Size of gig economy (% of GDP)     85.52   
9   United Arab Emirates  2019  Size of gig economy (% of GDP)     83.82   
10             Australia  2019  Size of gig economy (% of GDP)     82.16   
11               Germany  2019  Size of gig economy (% of GDP)     82.07   
12                 Qatar  2019  Size of gig economy (% of GDP)     74.83   
13           New Zealand  2019  Size of gig economy (% of GDP)     73.86   
14            Azerbaijan  2019  Size of gig economy (% of GDP)     73.58   

    new_rank_score  higher_is_better           Sub-Pillar  
0         5.990000              True  Technology Adoption  
1         5.722037              True  Technology Adoption  
2         5.680620              True  Technology Adoption  
3         5.507467              True  Technology Adoption  
4         5.500481              True  Technology Adoption  
5         5.464054              True  Technology Adoption  
6         5.412158              True  Technology Adoption  
7         5.294893              True  Technology Adoption  
8         5.267448              True  Technology Adoption  
9         5.182618              True  Technology Adoption  
10        5.099784              True  Technology Adoption  
11        5.095293              True  Technology Adoption  
12        4.734017              True  Technology Adoption  
13        4.685614              True  Technology Adoption  
14        4.671642              True  Technology Adoption

In [77]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

## 9. Size of digital economy (% of transactions)


In [78]:
indicator = indicators[8]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Size of digital economy (% of transactions)
size_digital_economy


In [79]:
df

Order        Country Name  Value   Score
0      1.0           Singapore  78.13  100.00
1      2.0         Switzerland  64.57   82.59
2      3.0         Korea, Rep.  63.66   81.42
3      4.0             Germany  61.45   78.58
4      5.0             Hungary  59.72   76.36
..     ...                 ...    ...     ...
129    NaN    Congo, Dem. Rep.    NaN     NaN
130    NaN  Dominican Republic    NaN     NaN
131    NaN              Guinea    NaN     NaN
132    NaN             Lesotho    NaN     NaN
133    NaN                Mali    NaN     NaN

[134 rows x 4 columns]

In [80]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Technology Adoption


In [81]:
# create standard columns
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [82]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.head(15)

Country Name  Year                                    Indicator  \
0        Singapore  2019  Size of digital economy (% of transactions)   
1      Switzerland  2019  Size of digital economy (% of transactions)   
2      Korea, Rep.  2019  Size of digital economy (% of transactions)   
3          Germany  2019  Size of digital economy (% of transactions)   
4          Hungary  2019  Size of digital economy (% of transactions)   
5            Japan  2019  Size of digital economy (% of transactions)   
6          Ireland  2019  Size of digital economy (% of transactions)   
7          Denmark  2019  Size of digital economy (% of transactions)   
8            Qatar  2019  Size of digital economy (% of transactions)   
9           Sweden  2019  Size of digital economy (% of transactions)   
10  Czech Republic  2019  Size of digital economy (% of transactions)   
11        Slovakia  2019  Size of digital economy (% of transactions)   
12          France  2019  Size of digital economy (% of transactions)   
13         Belgium  2019  Size of digital economy (% of transactions)   
14     Netherlands  2019  Size of digital economy (% of transactions)   

    data_col  new_rank_score  higher_is_better           Sub-Pillar  
0      78.13        4.898687              True  Technology Adoption  
1      64.57        4.222043              True  Technology Adoption  
2      63.66        4.176634              True  Technology Adoption  
3      61.45        4.066355              True  Technology Adoption  
4      59.72        3.980028              True  Technology Adoption  
5      56.21        3.804879              True  Technology Adoption  
6      54.35        3.712065              True  Technology Adoption  
7      54.22        3.705578              True  Technology Adoption  
8      54.17        3.703083              True  Technology Adoption  
9      53.01        3.645199              True  Technology Adoption  
10     52.40        3.614760              True  Technology Adoption  
11     51.38        3.563862              True  Technology Adoption  
12     50.08        3.498992              True  Technology Adoption  
13     49.51        3.470549              True  Technology Adoption  
14     48.86        3.438114              True  Technology Adoption

In [83]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

## 10. Venture Capital Availability


In [84]:
indicator = indicators[9]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Venture Capital Availability
TCdata360


In [85]:
df.head(15)

2007  2008  2009  2010  2011  2012  2013  2014  2015  2016  ...  \
0    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
1    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
2    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
7    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
8    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
9    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
10   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
11   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
12   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
13   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
14   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

    2008-2009  2009-2010   2010-2011   2011-2012 2012-2013   2013-2014  \
0         NaN        NaN   40.900000   54.127067       NaN   40.920500   
1         NaN        NaN   66.000000   49.000000       NaN   87.000000   
2         NaN        NaN    3.858249    3.165548       NaN    2.823529   
3         NaN        NaN  127.000000  142.000000       NaN  148.000000   
4         NaN        NaN    2.797549    2.242699       NaN    2.028571   
5         NaN        NaN  130.000000  142.000000       NaN  148.000000   
6         NaN        NaN    2.196592    2.090002       NaN    2.030303   
7         NaN        NaN  138.000000  141.000000       NaN  148.000000   
8         NaN        NaN    2.793415    2.468175       NaN    2.352941   
9         NaN        NaN  115.000000  127.000000       NaN  139.000000   
10        NaN        NaN    1.399599    1.550230       NaN    1.666667   
11        NaN        NaN  107.000000         NaN       NaN  107.000000   
12        NaN        NaN    2.071660    2.285828       NaN    2.941176   
13        NaN        NaN  136.000000  138.000000       NaN  125.000000   
14        NaN        NaN    3.001579    3.053193       NaN    3.382353   

     2014-2015 2015-2016  2016-2017  2017-2018  
0    38.903010       NaN        NaN        NaN  
1    92.000000       NaN        NaN        NaN  
2     2.601164       NaN        NaN        NaN  
3   144.000000       NaN        NaN        NaN  
4     2.177586       NaN        NaN        NaN  
5   144.000000       NaN        NaN        NaN  
6     2.249658       NaN        NaN        NaN  
7   141.000000       NaN        NaN        NaN  
8     2.260678       NaN        NaN        NaN  
9   138.000000       NaN        NaN        NaN  
10         NaN       NaN        NaN        NaN  
11         NaN       NaN        NaN        NaN  
12    2.682389       NaN        NaN        NaN  
13  121.000000       NaN        NaN        NaN  
14    3.295356       NaN        NaN        NaN  

[15 rows x 29 columns]

In [86]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Financing Incentives


In [87]:
df = df[(df.Indicator == 'Venture capital availability, 1-7 (best)')]
df = df[(df['Subindicator Type'] == '1-7 Best')]
df

2007      2008      2009      2010      2011      2012      2013  \
226         NaN       NaN       NaN       NaN  1.802891  1.494084       NaN   
512    2.696235  2.665708  2.524466  2.321898  2.145235  1.956589  1.844924   
796    4.491447  4.544287  4.296279  3.881013  3.716301  3.973436  4.143612   
1080   2.893384  2.839896  2.425063  2.131687  1.897883  1.909561  1.822370   
1366   2.404752  2.123574  1.999757  1.921578  1.789272  2.136096  2.369044   
...         ...       ...       ...       ...       ...       ...       ...   
41962  3.334126  3.115717  3.177300  3.044030  2.667287  2.311127  2.290705   
42246       NaN       NaN       NaN       NaN       NaN  2.587327  2.262899   
42530  3.770429  3.719011  3.890964  3.374518  3.011638  2.932891  3.054382   
42816  1.494845  1.984663  2.526940  2.298351  1.992631  2.073739  2.489051   
43102  2.634554  2.597143  2.581603  1.991631  1.735018  1.835783  1.794334   

           2014      2015      2016  ...  2008-2009  2009-2010  2010-2011  \
226    2.117647  2.348170       NaN  ...        NaN        NaN        NaN   
512    1.859132  1.948528  1.898624  ...        NaN        NaN        NaN   
796    4.123291  4.352251  4.352251  ...        NaN        NaN        NaN   
1080   1.745999  1.779793  2.009052  ...        NaN        NaN        NaN   
1366   2.425793  2.355755  2.537145  ...        NaN        NaN        NaN   
...         ...       ...       ...  ...        ...        ...        ...   
41962  2.565255  2.690639  2.968303  ...        NaN        NaN        NaN   
42246  1.876288  1.692528       NaN  ...        NaN        NaN        NaN   
42530  3.294088  3.185590  2.962898  ...        NaN        NaN        NaN   
42816  2.538825  2.372437  2.225287  ...        NaN        NaN        NaN   
43102  1.787351  1.619521  1.545050  ...        NaN        NaN        NaN   

      2011-2012 2012-2013  2013-2014 2014-2015 2015-2016  2016-2017  2017-2018  
226         NaN       NaN        NaN       NaN       NaN        NaN        NaN  
512         NaN       NaN        NaN       NaN       NaN        NaN        NaN  
796         NaN       NaN        NaN       NaN       NaN        NaN        NaN  
1080        NaN       NaN        NaN       NaN       NaN        NaN        NaN  
1366        NaN       NaN        NaN       NaN       NaN        NaN        NaN  
...         ...       ...        ...       ...       ...        ...        ...  
41962       NaN       NaN        NaN       NaN       NaN        NaN        NaN  
42246       NaN       NaN        NaN       NaN       NaN        NaN        NaN  
42530       NaN       NaN        NaN       NaN       NaN        NaN        NaN  
42816       NaN       NaN        NaN       NaN       NaN        NaN        NaN  
43102       NaN       NaN        NaN       NaN       NaN        NaN        NaN  

[152 rows x 29 columns]

In [88]:
# create standard columns
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['2019'] 
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_b(row,old_min=1,old_max=7))

In [89]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

Country Name  Year                     Indicator  data_col  \
226                  Angola  2019  Venture Capital Availability  1.705738   
512                 Albania  2019  Venture Capital Availability  3.014087   
796    United Arab Emirates  2019  Venture Capital Availability  4.832029   
1080              Argentina  2019  Venture Capital Availability  2.328207   
1366                Armenia  2019  Venture Capital Availability  3.311092   
...                     ...   ...                           ...       ...   
41962               Vietnam  2019  Venture Capital Availability  3.267500   
42246           Yemen, Rep.  2019  Venture Capital Availability  2.163688   
42530          South Africa  2019  Venture Capital Availability  3.051583   
42816                Zambia  2019  Venture Capital Availability  1.803466   
43102              Zimbabwe  2019  Venture Capital Availability  2.058023   

       new_rank_score  higher_is_better            Sub-Pillar  
226          1.586939              True  Financing Incentives  
512          2.675049              True  Financing Incentives  
796          4.186971              True  Financing Incentives  
1080         2.104625              True  Financing Incentives  
1366         2.922058              True  Financing Incentives  
...               ...               ...                   ...  
41962        2.885804              True  Financing Incentives  
42246        1.967801              True  Financing Incentives  
42530        2.706233              True  Financing Incentives  
42816        1.668216              True  Financing Incentives  
43102        1.879922              True  Financing Incentives  

[152 rows x 7 columns]

In [90]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

In [91]:
## 11. Doing Business Index

In [92]:
indicator = indicators[10]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Doing Business Index
doing_bus_idx


In [93]:
df

Country code      Economy              Region Income group  DB Year  \
0             AFG  Afghanistan          South Asia   Low income     2005   
1             AFG  Afghanistan          South Asia   Low income     2006   
2             AFG  Afghanistan          South Asia   Low income     2007   
3             AFG  Afghanistan          South Asia   Low income     2008   
4             AFG  Afghanistan          South Asia   Low income     2009   
...           ...          ...                 ...          ...      ...   
3020          ZWE     Zimbabwe  Sub-Saharan Africa   Low income     2015   
3021          ZWE     Zimbabwe  Sub-Saharan Africa   Low income     2016   
3022          ZWE     Zimbabwe  Sub-Saharan Africa   Low income     2017   
3023          ZWE     Zimbabwe  Sub-Saharan Africa   Low income     2018   
3024          ZWE     Zimbabwe  Sub-Saharan Africa   Low income     2019   

      Ease of doing business rank (DB19)  \
0                                    NaN   
1                                    NaN   
2                                    NaN   
3                                    NaN   
4                                    NaN   
...                                  ...   
3020                                 NaN   
3021                                 NaN   
3022                                 NaN   
3023                                 NaN   
3024                               155.0   

      Ease of doing business score (DB17-19 methodology)  \
0                                                   NaN    
1                                                   NaN    
2                                                   NaN    
3                                                   NaN    
4                                                   NaN    
...                                                 ...    
3020                                                NaN    
3021                                              47.74    
3022                                              47.73    
3023                                              48.52    
3024                                              50.44    

      Ease of doing business score (DB16 methodology)  \
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
...                                               ...   
3020                                            47.11   
3021                                            47.94   
3022                                              NaN   
3023                                              NaN   
3024                                              NaN   

      Ease of doing business score (DB15 methodology)  \
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
...                                               ...   
3020                                            44.36   
3021                                              NaN   
3022                                              NaN   
3023                                              NaN   
3024                                              NaN   

      Ease of doing business score  (DB10-14 methodology)  ...  \
0                                                   NaN    ...   
1                                                   NaN    ...   
2                                                   NaN    ...   
3                                                   NaN    ...   
4                                                   NaN    ...   
...                                                 ...    ...   
3020      

In [94]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Startup Environment


In [95]:
df = df[(df['DB Year'] == 2019)]
df.head(15)

Country code                Economy                      Region  \
14           AFG            Afghanistan                  South Asia   
30           ALB                Albania       Europe & Central Asia   
46           DZA                Algeria  Middle East & North Africa   
62           AGO                 Angola          Sub-Saharan Africa   
76           ATG    Antigua and Barbuda   Latin America & Caribbean   
92           ARG              Argentina   Latin America & Caribbean   
108          ARM                Armenia       Europe & Central Asia   
124          AUS              Australia           High income: OECD   
140          AUT                Austria           High income: OECD   
156          AZE             Azerbaijan       Europe & Central Asia   
168          BHS           Bahamas, The   Latin America & Caribbean   
180          BHR                Bahrain  Middle East & North Africa   
186         BANG             Bangladesh                  South Asia   
192     BGD_Chit  Bangladesh Chittagong                  South Asia   
208          BGD       Bangladesh Dhaka                  South Asia   

            Income group  DB Year  Ease of doing business rank (DB19)  \
14            Low income     2019                               167.0   
30   Upper middle income     2019                                63.0   
46   Upper middle income     2019                               157.0   
62   Lower middle income     2019                               173.0   
76           High income     2019                               112.0   
92           High income     2019                               119.0   
108  Upper middle income     2019                                41.0   
124          High income     2019                                18.0   
140          High income     2019                                26.0   
156  Upper middle income     2019                                25.0   
168          High income     2019                               118.0   
180          High income     2019                                62.0   
186  Lower middle income     2019                               176.0   
192  Lower middle income     2019                                 NaN   
208  Lower middle income     2019                                 NaN   

     Ease of doing business score (DB17-19 methodology)  \
14                                               47.77    
30                                               69.51    
46                                               49.65    
62                                               43.86    
76                                               59.48    
92                                               58.80    
108                                              75.37    
124                                              80.13    
140                                              78.57    
156                                              78.64    
168                                              58.90    
180                                              69.85    
186                                              41.97    
192                                              42.89    
208                                              41.71    

     Ease of doing business score (DB16 methodology)  \
14                                               NaN   
30                                               NaN   
46                                               NaN   
62                                               NaN   
76                                               NaN   
92                                               NaN   
108                                              NaN   
124                                              NaN   
140                                              NaN   
156                                              NaN   
168                                              NaN   
180                                              NaN   
186                      

In [96]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df.iloc[:,6]
df['Year'] = df.iloc[:,4]
df['Country Name'] = df.iloc[:,1]
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

<ipython-input-96-14f778ed67f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['higher_is_better'] = False
<ipython-input-96-14f778ed67f9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Indicator'] = indicator
<ipython-input-96-14f778ed67f9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [97]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.head(15)

Country Name  Year             Indicator  data_col  \
14             Afghanistan  2019  Doing Business Index     47.77   
30                 Albania  2019  Doing Business Index     69.51   
46                 Algeria  2019  Doing Business Index     49.65   
62                  Angola  2019  Doing Business Index     43.86   
76     Antigua and Barbuda  2019  Doing Business Index     59.48   
92               Argentina  2019  Doing Business Index     58.80   
108                Armenia  2019  Doing Business Index     75.37   
124              Australia  2019  Doing Business Index     80.13   
140                Austria  2019  Doing Business Index     78.57   
156             Azerbaijan  2019  Doing Business Index     78.64   
168           Bahamas, The  2019  Doing Business Index     58.90   
180                Bahrain  2019  Doing Business Index     69.85   
186             Bangladesh  2019  Doing Business Index     41.97   
192  Bangladesh Chittagong  2019  Doing Business Index     42.89   
208       Bangladesh Dhaka  2019  Doing Business Index     41.71   

     new_rank_score  higher_is_better           Sub-Pillar  
14         3.383723             False  Startup Environment  
30         4.468549             False  Startup Environment  
46         3.477535             False  Startup Environment  
62         3.188614             False  Startup Environment  
76         3.968052             False  Startup Environment  
92         3.934120             False  Startup Environment  
108        4.760963             False  Startup Environment  
124        4.998487             False  Startup Environment  
140        4.920643             False  Startup Environment  
156        4.924136             False  Startup Environment  
168        3.939110             False  Startup Environment  
180        4.485515             False  Startup Environment  
186        3.094303             False  Startup Environment  
192        3.140211             False  Startup Environment  
208        3.081329             False  Startup Environment

In [98]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

## 12. Strength of Legal Rights 

In [99]:
indicator = indicators[11]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Strength of Legal Rights
legal_rights_strength


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Series Name    268 non-null    object
 1   Series Code    266 non-null    object
 2   Country Name   266 non-null    object
 3   Country Code   266 non-null    object
 4   1990 [YR1990]  266 non-null    object
 5   2000 [YR2000]  266 non-null    object
 6   2011 [YR2011]  266 non-null    object
 7   2012 [YR2012]  266 non-null    object
 8   2013 [YR2013]  266 non-null    object
 9   2014 [YR2014]  266 non-null    object
 10  2015 [YR2015]  266 non-null    object
 11  2016 [YR2016]  266 non-null    object
 12  2017 [YR2017]  266 non-null    object
 13  2018 [YR2018]  266 non-null    object
 14  2019 [YR2019]  266 non-null    object
 15  2020 [YR2020]  266 non-null    object
dtypes: object(16)
memory usage: 34.0+ KB


In [101]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Startup Environment


#### Find Relevant Columns

In [102]:
df['Series Name'].unique()

array(['Strength of legal rights index', nan,
       'Data from database: World Development Indicators',
       'Last Updated: 06/30/2021'], dtype=object)

In [103]:
# remove unwanted rows
df = df[~df['Series Code'].isna()]
df = df.replace('..', np.nan)

In [104]:
df.loc[0][0]

'Strength of legal rights index'

In [105]:
df.head()

Series Name     Series Code  \
0  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
1  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
2  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
3  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
4  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   

     Country Name Country Code  1990 [YR1990]  2000 [YR2000]  2011 [YR2011]  \
0     Afghanistan          AFG            NaN            NaN            NaN   
1         Albania          ALB            NaN            NaN            NaN   
2         Algeria          DZA            NaN            NaN            NaN   
3  American Samoa          ASM            NaN            NaN            NaN   
4         Andorra          AND            NaN            NaN            NaN   

   2012 [YR2012] 2013 [YR2013] 2014 [YR2014] 2015 [YR2015] 2016 [YR2016]  \
0            NaN             9             9             9             9   
1            NaN             7             6             6             6   
2            NaN             2             2             2             2   
3            NaN           NaN           NaN           NaN           NaN   
4            NaN           NaN           NaN           NaN           NaN   

  2017 [YR2017] 2018 [YR2018] 2019 [YR2019]  2020 [YR2020]  
0             9            10            10            NaN  
1             8             8             8            NaN  
2             2             2             2            NaN  
3           NaN           NaN           NaN            NaN  
4           NaN           NaN           NaN            NaN

In [106]:
df.columns

Index(['Series Name', 'Series Code', 'Country Name', 'Country Code',
       '1990 [YR1990]', '2000 [YR2000]', '2011 [YR2011]', '2012 [YR2012]',
       '2013 [YR2013]', '2014 [YR2014]', '2015 [YR2015]', '2016 [YR2016]',
       '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]'],
      dtype='object')

In [107]:
# clean data
df = df.replace('..', np.nan)

df[['1990 [YR1990]', '2000 [YR2000]', '2011 [YR2011]', '2012 [YR2012]',
       '2013 [YR2013]', '2014 [YR2014]', '2015 [YR2015]', '2016 [YR2016]',
       '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]']] = df[['1990 [YR1990]', '2000 [YR2000]', '2011 [YR2011]', '2012 [YR2012]',
       '2013 [YR2013]', '2014 [YR2014]', '2015 [YR2015]', '2016 [YR2016]',
       '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]']].astype(float)

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 0 to 265
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series Name    266 non-null    object 
 1   Series Code    266 non-null    object 
 2   Country Name   266 non-null    object 
 3   Country Code   266 non-null    object 
 4   1990 [YR1990]  0 non-null      float64
 5   2000 [YR2000]  0 non-null      float64
 6   2011 [YR2011]  0 non-null      float64
 7   2012 [YR2012]  0 non-null      float64
 8   2013 [YR2013]  236 non-null    float64
 9   2014 [YR2014]  236 non-null    float64
 10  2015 [YR2015]  237 non-null    float64
 11  2016 [YR2016]  237 non-null    float64
 12  2017 [YR2017]  237 non-null    float64
 13  2018 [YR2018]  238 non-null    float64
 14  2019 [YR2019]  238 non-null    float64
 15  2020 [YR2020]  0 non-null      float64
dtypes: float64(12), object(4)
memory usage: 43.4+ KB


In [109]:
# keep only most recent value
df = df[['Series Name', 'Series Code', 'Country Name', 'Country Code','2019 [YR2019]']]

df['higher_is_better'] = True
df['Indicator'] = df['Series Name']
df['data_col'] = df['2019 [YR2019]']
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

#### Convert Scales

In [110]:
# convert 0-12 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_c(row, old_min=0,old_max=12))

In [111]:
df.head(16)

Series Name     Series Code  \
0   Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
1   Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
2   Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
3   Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
4   Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
5   Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
6   Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
7   Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
8   Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
9   Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
10  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
11  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
12  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
13  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
14  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   
15  Strength of legal rights index (0=weak to 12=s...  IC.LGL.CRED.XQ   

           Country Name Country Code  2019 [YR2019]  higher_is_better  \
0           Afghanistan          AFG           10.0              True   
1               Albania          ALB            8.0              True   
2               Algeria          DZA            2.0              True   
3        American Samoa          ASM            NaN              True   
4               Andorra          AND            NaN              True   
5                Angola          AGO            1.0              True   
6   Antigua and Barbuda          ATG            5.0              True   
7             Argentina          ARG            2.0              True   
8               Armenia          ARM            6.0              True   
9                 Aruba          ABW            NaN              True   
10            Australia          AUS           11.0              True   
11              Austria          AUT            4.0              True   
12           Azerbaijan          AZE           12.0              True   
13         Bahamas, The          BHS            6.0              True   
14              Bahrain          BHR            3.0              True   
15           Bangladesh          BGD            5.0              True   

                                            Indicator  data_col  Year  \
0   Strength of legal rights index (0=weak to 12=s...      10.0  2019   
1   Strength of legal rights index (0=weak to 12=s...       8.0  2019   
2   Strength of legal rights index (0=weak to 12=s...       2.0  2019   
3   Strength of legal rights index (0=weak to 12=s...       NaN  2019   
4   Strength of legal rights index (0=weak to 12=s...       NaN  2019   
5   Strength of legal rights index (0=weak to 12=s...       1.0  2019   
6   Strength of legal rights index (0=weak to 12=s...       5.0  2019   
7   Strength of legal rights index (0=weak to 12=s...       2.0  2019   
8   Strength of legal rights index (0=weak to 12=s...       6.0  2019   
9   Strength of legal rights index (0=weak to 12=s...       NaN  2019   
10  Strength of legal rights index (0=weak to 12=s...      11.0  2019   
11  Strength of legal rights index (0=weak to 12=s...       4.0  2019   
12  Strength of legal rights index (0=weak to 12=s...      12.0  2019   
13  Strength of legal rights index (0=weak to 12=s...       6.0  2019   
14  Strength of legal rights index (0=weak to 12=s...       3.0  2019   
15  Strength of legal rights index (0=weak to 12=s...       5.0  2019   

             Sub-Pillar  new_rank_score  
0   Startup Environment        5.158333  
1   Startup Environment        4.326667  
2   Startup Environment        1.831667  
3   Startup Environment             NaN  
4   Startup Environment             NaN  
5   Startup Environment        1.415833  
6   Startup Environmen

In [112]:
df.columns

Index(['Series Name', 'Series Code', 'Country Name', 'Country Code',
       '2019 [YR2019]', 'higher_is_better', 'Indicator', 'data_col', 'Year',
       'Sub-Pillar', 'new_rank_score'],
      dtype='object')

#### Prepare Output

In [113]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.sort_values(by='new_rank_score', ascending=False).head(16)

Country Name  Year  \
12          Azerbaijan  2019   
132         Montenegro  2019   
141        New Zealand  2019   
158        Puerto Rico  2019   
28   Brunei Darussalam  2019   
119             Malawi  2019   
105             Kosovo  2019   
101              Kenya  2019   
162             Rwanda  2019   
42            Colombia  2019   
190         Tajikistan  2019   
10           Australia  2019   
99              Jordan  2019   
206      United States  2019   
209            Vanuatu  2019   
215             Zambia  2019   

                                             Indicator  data_col  \
12   Strength of legal rights index (0=weak to 12=s...      12.0   
132  Strength of legal rights index (0=weak to 12=s...      12.0   
141  Strength of legal rights index (0=weak to 12=s...      12.0   
158  Strength of legal rights index (0=weak to 12=s...      12.0   
28   Strength of legal rights index (0=weak to 12=s...      12.0   
119  Strength of legal rights index (0=weak to 12=s...      11.0   
105  Strength of legal rights index (0=weak to 12=s...      11.0   
101  Strength of legal rights index (0=weak to 12=s...      11.0   
162  Strength of legal rights index (0=weak to 12=s...      11.0   
42   Strength of legal rights index (0=weak to 12=s...      11.0   
190  Strength of legal rights index (0=weak to 12=s...      11.0   
10   Strength of legal rights index (0=weak to 12=s...      11.0   
99   Strength of legal rights index (0=weak to 12=s...      11.0   
206  Strength of legal rights index (0=weak to 12=s...      11.0   
209  Strength of legal rights index (0=weak to 12=s...      11.0   
215  Strength of legal rights index (0=weak to 12=s...      11.0   

     new_rank_score  higher_is_better           Sub-Pillar  
12         5.990000              True  Startup Environment  
132        5.990000              True  Startup Environment  
141        5.990000              True  Startup Environment  
158        5.990000              True  Startup Environment  
28         5.990000              True  Startup Environment  
119        5.574167              True  Startup Environment  
105        5.574167              True  Startup Environment  
101        5.574167              True  Startup Environment  
162        5.574167              True  Startup Environment  
42         5.574167              True  Startup Environment  
190        5.574167              True  Startup Environment  
10         5.574167              True  Startup Environment  
99         5.574167              True  Startup Environment  
206        5.574167              True  Startup Environment  
209        5.574167              True  Startup Environment  
215        5.574167              True  Startup Environment

In [114]:
# output scores
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

### 13. Time to start a business


#### Load Data

In [115]:
indicator = indicators[12]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Time to Start a Business
time_start_bus


In [116]:
# remove unwanted rows
df = df[~df['Series Code'].isna()]
df = df.replace('..', np.nan)

In [117]:
df[['1990 [YR1990]', '2000 [YR2000]', '2011 [YR2011]', '2012 [YR2012]',
       '2013 [YR2013]', '2014 [YR2014]', '2015 [YR2015]', '2016 [YR2016]',
       '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]']] = df[['1990 [YR1990]', '2000 [YR2000]', '2011 [YR2011]', '2012 [YR2012]',
       '2013 [YR2013]', '2014 [YR2014]', '2015 [YR2015]', '2016 [YR2016]',
       '2017 [YR2017]', '2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]']].astype(float)

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 0 to 265
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series Name    266 non-null    object 
 1   Series Code    266 non-null    object 
 2   Country Name   266 non-null    object 
 3   Country Code   266 non-null    object 
 4   1990 [YR1990]  0 non-null      float64
 5   2000 [YR2000]  0 non-null      float64
 6   2011 [YR2011]  221 non-null    float64
 7   2012 [YR2012]  225 non-null    float64
 8   2013 [YR2013]  236 non-null    float64
 9   2014 [YR2014]  236 non-null    float64
 10  2015 [YR2015]  237 non-null    float64
 11  2016 [YR2016]  237 non-null    float64
 12  2017 [YR2017]  237 non-null    float64
 13  2018 [YR2018]  238 non-null    float64
 14  2019 [YR2019]  238 non-null    float64
 15  2020 [YR2020]  0 non-null      float64
dtypes: float64(12), object(4)
memory usage: 35.3+ KB


In [119]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Startup Environment


In [120]:
# keep only most recent value
df = df[['Series Name', 'Series Code', 'Country Name', 'Country Code','2019 [YR2019]']]

df['higher_is_better'] = False 
df['Indicator'] = df['Series Name']
df['data_col'] = df['2019 [YR2019]']
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

In [121]:
first = df['data_col'].quantile(1/5)
second = df['data_col'].quantile(2/5)
third = df['data_col'].quantile(3/5)
fourth = df['data_col'].quantile(4/5)
fifth = df['data_col'].quantile(5/5)
data = {'Min':min_rank,'Max':max_rank,'1':first,'2':second,'3':third,'4':fourth,'5':fifth}
sextile = pd.DataFrame(data, index=[0])
sextile

Min    Max    1     2     3          4      5
0  0.5  230.0  7.0  11.5  17.1  25.553846  230.0

In [122]:
df.head()

Series Name  Series Code    Country Name  \
0  Time required to start a business (days)  IC.REG.DURS     Afghanistan   
1  Time required to start a business (days)  IC.REG.DURS         Albania   
2  Time required to start a business (days)  IC.REG.DURS         Algeria   
3  Time required to start a business (days)  IC.REG.DURS  American Samoa   
4  Time required to start a business (days)  IC.REG.DURS         Andorra   

  Country Code  2019 [YR2019]  higher_is_better  \
0          AFG            8.5             False   
1          ALB            4.5             False   
2          DZA           18.0             False   
3          ASM            NaN             False   
4          AND            NaN             False   

                                  Indicator  data_col  Year  \
0  Time required to start a business (days)       8.5  2019   
1  Time required to start a business (days)       4.5  2019   
2  Time required to start a business (days)      18.0  2019   
3  Time required to start a business (days)       NaN  2019   
4  Time required to start a business (days)       NaN  2019   

            Sub-Pillar  
0  Startup Environment  
1  Startup Environment  
2  Startup Environment  
3  Startup Environment  
4  Startup Environment

In [123]:
df.loc[df['data_col'] <= first, 'new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_I(row, old_min=min_rank,old_max=first))
df.loc[(df['data_col'] > first) & (df['data_col'] <= second), 'new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_II(row, old_min=first,old_max=second))
df.loc[(df['data_col'] > second) & (df['data_col'] <= third), 'new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_III(row, old_min=second,old_max=third))
df.loc[(df['data_col'] > third) & (df['data_col'] <= fourth), 'new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_IV(row, old_min=third,old_max=fourth))
df.loc[df['data_col'] > fourth, 'new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_V(row, old_min=fourth,old_max=max_rank))

df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (5.99-row)+1)

df=df[df['new_rank_score'].notna()]
df

Series Name  Series Code  \
0    Time required to start a business (days)  IC.REG.DURS   
1    Time required to start a business (days)  IC.REG.DURS   
2    Time required to start a business (days)  IC.REG.DURS   
5    Time required to start a business (days)  IC.REG.DURS   
6    Time required to start a business (days)  IC.REG.DURS   
..                                        ...          ...   
261  Time required to start a business (days)  IC.REG.DURS   
262  Time required to start a business (days)  IC.REG.DURS   
263  Time required to start a business (days)  IC.REG.DURS   
264  Time required to start a business (days)  IC.REG.DURS   
265  Time required to start a business (days)  IC.REG.DURS   

                                   Country Name Country Code  2019 [YR2019]  \
0                                   Afghanistan          AFG       8.500000   
1                                       Albania          ALB       4.500000   
2                                       Algeria          DZA      18.000000   
5                                        Angola          AGO      36.000000   
6                           Antigua and Barbuda          ATG      19.000000   
..                                          ...          ...            ...   
261                          Sub-Saharan Africa          SSF      21.525000   
262  Sub-Saharan Africa (excluding high income)          SSA      21.302128   
263   Sub-Saharan Africa (IDA & IBRD countries)          TSS      21.525000   
264                         Upper middle income          UMC      22.751923   
265                                       World          WLD      19.597382   

     higher_is_better                                 Indicator   data_col  \
0               False  Time required to start a business (days)   8.500000   
1               False  Time required to start a business (days)   4.500000   
2               False  Time required to start a business (days)  18.000000   
5               False  Time required to start a business (days)  36.000000   
6               False  Time required to start a business (days)  19.000000   
..                ...                                       ...        ...   
261             False  Time required to start a business (days)  21.525000   
262             False  Time required to start a business (days)  21.302128   
263             False  Time required to start a business (days)  21.525000   
264             False  Time required to start a business (days)  22.751923   
265             False  Time required to start a business (days)  19.597382   

     Year           Sub-Pillar  new_rank_score  
0    2019  Startup Environment        4.660000  
1    2019  Startup Environment        5.380769  
2    2019  Startup Environment        2.884604  
5    2019  Startup Environment        1.939416  
6    2019  Startup Environment        2.767498  
..    ...                  ...             ...  
261  2019  Startup Environment        2.471804  
262  2019  Startup Environment        2.497904  
263  2019  Startup Environment        2.471804  
264  2019  Startup Environment        2.328123  
265  2019  Startup Environment        2.697540  

[238 rows x 11 columns]

In [124]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.sort_values(by='new_rank_score', ascending=False).head(16)

Country Name  Year                                 Indicator  \
141           New Zealand  2019  Time required to start a business (days)   
72                Georgia  2019  Time required to start a business (days)   
171             Singapore  2019  Time required to start a business (days)   
86   Hong Kong SAR, China  2019  Time required to start a business (days)   
35                 Canada  2019  Time required to start a business (days)   
10              Australia  2019  Time required to start a business (days)   
194                  Togo  2019  Time required to start a business (days)   
251         North America  2019  Time required to start a business (days)   
208            Uzbekistan  2019  Time required to start a business (days)   
97                Jamaica  2019  Time required to start a business (days)   
12             Azerbaijan  2019  Time required to start a business (days)   
53                Denmark  2019  Time required to start a business (days)   
62                Estonia  2019  Time required to start a business (days)   
139           Netherlands  2019  Time required to start a business (days)   
204  United Arab Emirates  2019  Time required to start a business (days)   
76                 Greece  2019  Time required to start a business (days)   

     data_col  new_rank_score  higher_is_better           Sub-Pillar  
141      0.50        5.990000             False  Startup Environment  
72       1.00        5.913846             False  Startup Environment  
171      1.50        5.837692             False  Startup Environment  
86       1.50        5.837692             False  Startup Environment  
35       1.50        5.837692             False  Startup Environment  
10       2.00        5.761538             False  Startup Environment  
194      2.50        5.685385             False  Startup Environment  
251      2.85        5.632077             False  Startup Environment  
208      3.00        5.609231             False  Startup Environment  
97       3.00        5.609231             False  Startup Environment  
12       3.50        5.533077             False  Startup Environment  
53       3.50        5.533077             False  Startup Environment  
62       3.50        5.533077             False  Startup Environment  
139      3.50        5.533077             False  Startup Environment  
204      3.80        5.487385             False  Startup Environment  
76       4.00        5.456923             False  Startup Environment

#### Prepare Output

In [125]:
bf

'time_start_bus'

In [126]:
# output scores
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator),index = False)
df.to_csv('../non-index/business_{}_scores.csv'.format(indicator),index = False)

### 14. Ease doing business


#### Load Data

In [127]:
indicator = indicators[13]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Ease of Doing Business
ease_doing_bus


In [128]:
# remove unwanted rows
df = df.replace('..', np.nan)
df = df[~df['Series Code'].isna()]


In [129]:
df['Series Name'].unique()

array(['Ease of doing business index (1=most business-friendly regulations)'],
      dtype=object)

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 0 to 265
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series Name    266 non-null    object 
 1   Series Code    266 non-null    object 
 2   Country Name   266 non-null    object 
 3   Country Code   266 non-null    object 
 4   1990 [YR1990]  0 non-null      float64
 5   2000 [YR2000]  0 non-null      float64
 6   2011 [YR2011]  0 non-null      float64
 7   2012 [YR2012]  0 non-null      float64
 8   2013 [YR2013]  0 non-null      float64
 9   2014 [YR2014]  0 non-null      float64
 10  2015 [YR2015]  0 non-null      float64
 11  2016 [YR2016]  0 non-null      float64
 12  2017 [YR2017]  0 non-null      float64
 13  2018 [YR2018]  0 non-null      float64
 14  2019 [YR2019]  189 non-null    object 
 15  2020 [YR2020]  0 non-null      float64
dtypes: float64(11), object(5)
memory usage: 35.3+ KB


In [131]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Startup Environment


In [132]:
# keep only most recent value
df = df[['Series Name', 'Series Code', 'Country Name', 'Country Code','2019 [YR2019]']]

# df['higher_is_better'] = False
df['Indicator'] = df['Series Name']
df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
df['data_col'] = df['2019 [YR2019]']
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

# df['data_norm'] = df['data_norm'] = (df.data_col - df.data_col.mean())/df.data_col.std()

In [133]:
rank_min = df.data_col.min()
rank_max = df.data_col.max()

In [134]:
rank_min, rank_max

(1.0, 190.0)

In [135]:
# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=rank_min,old_max=rank_max))

In [136]:
# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (5.99-row)+1)

In [137]:
df.sort_values(by='new_rank_score', ascending=False).head(16)

Series Name     Series Code  \
141  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
171  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
86   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
53   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
104  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
206  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
72   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
205  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
147  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
187  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
115  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
120  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
126  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
10   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
204  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
145  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   

             Country Name Country Code  2019 [YR2019]  \
141           New Zealand          NZL            1.0   
171             Singapore          SGP            2.0   
86   Hong Kong SAR, China          HKG            3.0   
53                Denmark          DNK            4.0   
104           Korea, Rep.          KOR            5.0   
206         United States          USA            6.0   
72                Georgia          GEO            7.0   
205        United Kingdom          GBR            8.0   
147                Norway          NOR            9.0   
187                Sweden          SWE           10.0   
115             Lithuania          LTU           11.0   
120              Malaysia          MYS           12.0   
126             Mauritius          MUS           13.0   
10              Australia          AUS           14.0   
204  United Arab Emirates          ARE           16.0   
145       North Macedonia          MKD           17.0   

                                             Indicator  data_col  Year  \
141  Ease of doing business index (1=most business-...       1.0  2019   
171  Ease of doing business index (1=most business-...       2.0  2019   
86   Ease of doing business index (1=most business-...       3.0  2019   
53   Ease of doing business index (1=most business-...       4.0  2019   
104  Ease of doing business index (1=most business-...       5.0  2019   
206  Ease of doing business index (1=most business-...       6.0  2019   
72   Ease of doing business index (1=most business-...       7.0  2019   
205  Ease of doing business index (1=most business-...       8.0  2019   
147  Ease of doing business index (1=most business-...       9.0  2019   
187  Ease of doing business index (1=most business-...      10.0  2019   
115  Ease of doing business index (1=most business-...      11.0  2019   
120  Ease of doing business index (1=most business-...      12.0  2019   
126  Ease of doing business index (1=most business-...      13.0  2019   
10   Ease of doing business index (1=most business-...      14.0  2019   
204  Ease of doing business index (1=most business-...      16.0  2019   
145  Ease of doing business index (1=most business-...      17.0  2019   

              Sub-Pillar  new_rank_score  
141  Startup Environment        5.990000  
171  Startup Environment        5.963598  
86   Startup Environment        5.937196  
53   Startup Environment        5.910794  
104  Startup Environment        5.884392  
206  Startup Environment        5.857989  
72   Startup Environment        5.831587  
205  Startup Environment        5.805185  
147  Startup Environment        5.778783  
187  Startup Environment        5.752381  
115  Startup Environment        5.725979  
120  Startup Environment        5.69957

In [138]:
df['higher_is_better'] = True
df.head(15)

Series Name     Series Code  \
0   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
1   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
2   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
3   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
4   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
5   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
6   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
7   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
8   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
9   Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
10  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
11  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
12  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
13  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   
14  Ease of doing business index (1=most business-...  IC.BUS.EASE.XQ   

           Country Name Country Code  2019 [YR2019]  \
0           Afghanistan          AFG          173.0   
1               Albania          ALB           82.0   
2               Algeria          DZA          157.0   
3        American Samoa          ASM            NaN   
4               Andorra          AND            NaN   
5                Angola          AGO          177.0   
6   Antigua and Barbuda          ATG          113.0   
7             Argentina          ARG          126.0   
8               Armenia          ARM           47.0   
9                 Aruba          ABW            NaN   
10            Australia          AUS           14.0   
11              Austria          AUT           27.0   
12           Azerbaijan          AZE           34.0   
13         Bahamas, The          BHS          119.0   
14              Bahrain          BHR           43.0   

                                            Indicator  data_col  Year  \
0   Ease of doing business index (1=most business-...     173.0  2019   
1   Ease of doing business index (1=most business-...      82.0  2019   
2   Ease of doing business index (1=most business-...     157.0  2019   
3   Ease of doing business index (1=most business-...       NaN  2019   
4   Ease of doing business index (1=most business-...       NaN  2019   
5   Ease of doing business index (1=most business-...     177.0  2019   
6   Ease of doing business index (1=most business-...     113.0  2019   
7   Ease of doing business index (1=most business-...     126.0  2019   
8   Ease of doing business index (1=most business-...      47.0  2019   
9   Ease of doing business index (1=most business-...       NaN  2019   
10  Ease of doing business index (1=most business-...      14.0  2019   
11  Ease of doing business index (1=most business-...      27.0  2019   
12  Ease of doing business index (1=most business-...      34.0  2019   
13  Ease of doing business index (1=most business-...     119.0  2019   
14  Ease of doing business index (1=most business-...      43.0  2019   

             Sub-Pillar  new_rank_score  higher_is_better  
0   Startup Environment        1.448836              True  
1   Startup Environment        3.851429              True  
2   Startup Environment        1.871270              True  
3   Startup Environment             NaN              True  
4   Startup Environment             NaN              True  
5   Startup Environment        1.343228              True  
6   Startup Environment        3.032963              True  
7   Startup Environment        2.689735              True  
8   Startup Environment        4.775503              True  
9   Startup Environment             NaN              True  
10  Startup Environment        5.646772              True  
11  Startup Environment        5.303545              True  
12  Startup Environment        5.118730    

#### Prepare Output

In [139]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]

# output scores


## 15. Ease of finding skilled employees

In [140]:
indicator = indicators[14]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Ease of finding skilled employees
ease_of_finding_skilled_employees


In [141]:
df.head()

2017  2018  2019    Country  Unnamed: 4
0     3.88  4.03  3.89    Albania         NaN
1     3.98  3.84  4.12    Algeria         NaN
2  No data  2.08  2.76     Angola         NaN
3     4.35   4.1  4.19  Argentina         NaN
4     3.82  3.87  4.03    Armenia         NaN

In [142]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Startup Environment


In [143]:
values = ['2017','2018','2019']

In [144]:
df = df.replace('No data', np.nan)

In [145]:
df[values] = df[values].astype(float)

In [146]:
df.head()

2017  2018  2019    Country  Unnamed: 4
0  3.88  4.03  3.89    Albania         NaN
1  3.98  3.84  4.12    Algeria         NaN
2   NaN  2.08  2.76     Angola         NaN
3  4.35  4.10  4.19  Argentina         NaN
4  3.82  3.87  4.03    Armenia         NaN

In [147]:
df[values].describe()

2017        2018        2019
count  132.000000  136.000000  137.000000
mean     4.183258    4.136176    4.196058
std      0.664458    0.659186    0.589124
min      2.720000    2.080000    2.760000
25%      3.685000    3.637500    3.840000
50%      4.065000    4.095000    4.170000
75%      4.692500    4.655000    4.630000
max      5.670000    5.750000    5.320000

In [148]:
# create standard columns
df['data_col'] = df['2019']
df['new_rank_score'] = df['data_col']
df['higher_is_better'] = True
df['Indicator'] = indicator
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

df.rename(columns={'Country':'Country Name'}, inplace=True)


df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

Country Name  Year                          Indicator  data_col  \
0          Albania  2019  Ease of finding skilled employees      3.89   
1          Algeria  2019  Ease of finding skilled employees      4.12   
2           Angola  2019  Ease of finding skilled employees      2.76   
3        Argentina  2019  Ease of finding skilled employees      4.19   
4          Armenia  2019  Ease of finding skilled employees      4.03   
..             ...   ...                                ...       ...   
134  Venezuela, RB  2019  Ease of finding skilled employees      3.39   
135        Vietnam  2019  Ease of finding skilled employees      3.96   
136    Yemen, Rep.  2019  Ease of finding skilled employees      3.71   
137         Zambia  2019  Ease of finding skilled employees      4.76   
138       Zimbabwe  2019  Ease of finding skilled employees      4.38   

     new_rank_score  higher_is_better           Sub-Pillar  
0              3.89              True  Startup Environment  
1              4.12              True  Startup Environment  
2              2.76              True  Startup Environment  
3              4.19              True  Startup Environment  
4              4.03              True  Startup Environment  
..              ...               ...                  ...  
134            3.39              True  Startup Environment  
135            3.96              True  Startup Environment  
136            3.71              True  Startup Environment  
137            4.76              True  Startup Environment  
138            4.38              True  Startup Environment  

[139 rows x 7 columns]

In [149]:
# output scores to csv
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']].to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

## 16. Amount invested into startups yearly from private, public, blended sources (respectively)


In [150]:
indicator = indicators[15]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Amount invested into startups yearly from private, public, blended sources (respectively)
start_up_investment


In [151]:
df.head(15)

LOCATION    Country SUBJECT                      Subject STAGES  \
0       AUS  Australia  VC_INV  Venture capital investments   VC_T   
1       AUS  Australia  VC_INV  Venture capital investments   VC_T   
2       AUS  Australia  VC_INV  Venture capital investments   VC_T   
3       AUS  Australia  VC_INV  Venture capital investments   VC_T   
4       AUS  Australia  VC_INV  Venture capital investments   VC_T   
5       AUS  Australia  VC_INV  Venture capital investments   VC_T   
6       AUS  Australia  VC_INV  Venture capital investments   VC_T   
7       AUS  Australia  VC_INV  Venture capital investments   VC_T   
8       AUS  Australia  VC_INV  Venture capital investments   VC_T   
9       AUS  Australia  VC_INV  Venture capital investments   VC_T   
10      AUS  Australia  VC_INV  Venture capital investments   VC_T   
11      AUS  Australia  VC_INV  Venture capital investments   VC_T   
12      AUS  Australia  VC_INV  Venture capital investments   VC_T   
13      AUS  Australia  VC_INV  Venture capital investments   VC_T   
14      AUS  Australia  VC_INV  Venture capital investments   SEED   

   Development stages MEASURE              Measure  TIME  Year Unit Code  \
0               Total   USD_V  USD, current prices  2006  2006       USD   
1               Total   USD_V  USD, current prices  2007  2007       USD   
2               Total   USD_V  USD, current prices  2008  2008       USD   
3               Total   USD_V  USD, current prices  2009  2009       USD   
4               Total   USD_V  USD, current prices  2010  2010       USD   
5               Total   USD_V  USD, current prices  2011  2011       USD   
6               Total   USD_V  USD, current prices  2012  2012       USD   
7               Total   USD_V  USD, current prices  2013  2013       USD   
8               Total   USD_V  USD, current prices  2014  2014       USD   
9               Total   USD_V  USD, current prices  2015  2015       USD   
10              Total   USD_V  USD, current prices  2016  2016       USD   
11              Total   USD_V  USD, current prices  2017  2017       USD   
12              Total   USD_V  USD, current prices  2018  2018       USD   
13              Total   USD_V  USD, current prices  2019  2019       USD   
14               Seed   USD_V  USD, current prices  2006  2006       USD   

         Unit  PowerCode Code PowerCode  Reference Period Code  \
0   US Dollar               6  Millions                    NaN   
1   US Dollar               6  Millions                    NaN   
2   US Dollar               6  Millions                    NaN   
3   US Dollar               6  Millions                    NaN   
4   US Dollar               6  Millions                    NaN   
5   US Dollar               6  Millions                    NaN   
6   US Dollar               6  Millions                    NaN   
7   US Dollar               6  Millions                    NaN   
8   US Dollar               6  Millions                    NaN   
9   US Dollar               6  Millions                    NaN   
10  US Dollar               6  Millions                    NaN   
11  US Dollar               6  Millions                    NaN   
12  US Dollar               6  Millions                    NaN   
13  US Dollar               6  Millions                    NaN   
14  US Dollar               6  Millions                    NaN   

    Reference Period       Value  Flag Codes  Flags  
0                NaN  456.334579         NaN    NaN  
1                NaN  680.293170         NaN    NaN  
2                NaN  755.759626         NaN    NaN  
3                NaN  532.682779         NaN    NaN  
4                NaN  367.836251         NaN    NaN  
5                NaN  246.528233         NaN    NaN  
6                NaN  331.331196         NaN    NaN  
7                NaN  252.934084         NaN    NaN  
8                NaN  265.918369         NaN    NaN  
9                NaN  288.485377         NaN    NaN  
10            

In [152]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Startup Environment


In [153]:
df = df[(df['Development stages']=='Total') & (df.Year == 2019) & (df.MEASURE == 'USD_V')]
df.head(15)

LOCATION         Country SUBJECT                      Subject STAGES  \
13        AUS       Australia  VC_INV  Venture capital investments   VC_T   
120       AUT         Austria  VC_INV  Venture capital investments   VC_T   
232       BEL         Belgium  VC_INV  Venture capital investments   VC_T   
343       CAN          Canada  VC_INV  Venture capital investments   VC_T   
448       CZE  Czech Republic  VC_INV  Venture capital investments   VC_T   
538       DNK         Denmark  VC_INV  Venture capital investments   VC_T   
650       FIN         Finland  VC_INV  Venture capital investments   VC_T   
762       FRA          France  VC_INV  Venture capital investments   VC_T   
874       DEU         Germany  VC_INV  Venture capital investments   VC_T   
985       GRC          Greece  VC_INV  Venture capital investments   VC_T   
1080      HUN         Hungary  VC_INV  Venture capital investments   VC_T   
1188      IRL         Ireland  VC_INV  Venture capital investments   VC_T   
1298      ITA           Italy  VC_INV  Venture capital investments   VC_T   
1408      JPN           Japan  VC_INV  Venture capital investments   VC_T   
1473      KOR           Korea  VC_INV  Venture capital investments   VC_T   

     Development stages MEASURE              Measure  TIME  Year Unit Code  \
13                Total   USD_V  USD, current prices  2019  2019       USD   
120               Total   USD_V  USD, current prices  2019  2019       USD   
232               Total   USD_V  USD, current prices  2019  2019       USD   
343               Total   USD_V  USD, current prices  2019  2019       USD   
448               Total   USD_V  USD, current prices  2019  2019       USD   
538               Total   USD_V  USD, current prices  2019  2019       USD   
650               Total   USD_V  USD, current prices  2019  2019       USD   
762               Total   USD_V  USD, current prices  2019  2019       USD   
874               Total   USD_V  USD, current prices  2019  2019       USD   
985               Total   USD_V  USD, current prices  2019  2019       USD   
1080              Total   USD_V  USD, current prices  2019  2019       USD   
1188              Total   USD_V  USD, current prices  2019  2019       USD   
1298              Total   USD_V  USD, current prices  2019  2019       USD   
1408              Total   USD_V  USD, current prices  2019  2019       USD   
1473              Total   USD_V  USD, current prices  2019  2019       USD   

           Unit  PowerCode Code PowerCode  Reference Period Code  \
13    US Dollar               6  Millions                    NaN   
120   US Dollar               6  Millions                    NaN   
232   US Dollar               6  Millions                    NaN   
343   US Dollar               6  Millions                    NaN   
448   US Dollar               6  Millions                    NaN   
538   US Dollar               6  Millions                    NaN   
650   US Dollar               6  Millions                    NaN   
762   US Dollar               6  Millions                    NaN   
874   US Dollar               6  Millions                    NaN   
985   US Dollar               6  Millions                    NaN   
1080  US Dollar               6  Millions                    NaN   
1188  US Dollar               6  Millions                    NaN   
1298  US Dollar               6  Millions                    NaN   
1408  US Dollar               6  Millions                    NaN   
1473  US Dollar               6  Millions                    NaN   

      Reference Period        Value  Flag Codes  Flags  
13                 NaN   487.310802         NaN    NaN  
120                NaN    90.416881         NaN    NaN  
232                NaN   395.828165         NaN    NaN  
343                NaN  3286.872933         NaN    NaN  
448                NaN    27.204683         NaN    NaN  
538                NaN   352.437466         NaN    NaN  
650                NaN   321.566548

In [154]:
# create standard columns
df['Country Name'] = df['Country']
df['Indicator'] = indicator
df['data_col'] = df['Value']
df['higher_is_better'] = True
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

In [155]:
df = df[['Country Name', 'Year','Indicator','data_col','higher_is_better','Sub-Pillar']]

# Normalize country names as much as possible
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bahamas.*$)', 'Bahamas (the)')
df['Country Name'] = df['Country Name'].replace('Bahrain (Kingdom of)','Bahrain')
df['Country Name'] = df['Country Name'].replace('Bolivia','Bolivia (Plurinational State of)')
df['Country Name'] = df['Country Name'].replace('Bolivia, Plurinational State of','Bolivia (Plurinational State of)')
df['Country Name'] = df['Country Name'].replace('Brunei','Brunei Darussalam')
df['Country Name'] = df['Country Name'].replace('Bulgaria (Rep.)','Bulgaria')
df['Country Name'] = df['Country Name'].replace('Central African Republic','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace('Central African Rep.','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace("China (People's Rep.)",'China')
df['Country Name'] = df['Country Name'].replace("Comoros",'Comoros (the)')
df['Country Name'] = df['Country Name'].replace("Congo",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Brazzaville)",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Rep. of the)",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Democratic Republic of the)",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo, Dem. Rep.",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo, The Democratic Republic of the",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("DR Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Democratic Republic of Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Democratic Republic of the Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Dem. Rep. of the Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Cote d'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Côte d’Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cote D'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cote dIvoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].str.replace(r"(^.*Côte d'Ivoire.*$)", "Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cōte d'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Ivory Coast","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Dem. People's Rep. of Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("Democratic People's Republic of Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("Korea, Dem. People's Rep.","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("North Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Czech.*$)', 'Czechia')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Dominican Re.*$)', 'Dominican Republic (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Hong Kong.*$)', 'China, Hong Kong Special Administrative Region')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Hongkong.*$)', 'China, Hong Kong Special Administrative Region')
df['Country Name'] = df['Country Name'].replace("Eswatini (Kingdom of)",'Eswatini')
df['Country Name'] = df['Country Name'].replace("Swaziland",'Eswatini')
df['Country Name'] = df['Country Name'].replace("Faröe Islands",'Faroe Islands')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Gambia.*$)', 'Gambia (the)')
df['Country Name'] = df['Country Name'].replace("Georgia (Country)",'Georgia')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Iran.*$)', 'Iran (Islamic Republic of)')
df['Country Name'] = df['Country Name'].replace("Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Rep. of)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Rep.)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Republic of)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Korea, Rep.*$)', 'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea, South",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("South Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Republic of Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace('Republic of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kyrgyz.*$)', 'Kyrgyzstan')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Lao.*$)', "Lao People's Democratic Republic (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macao.*$)', "China, Macao Special Administrative Region")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macau.*$)', "China, Macao Special Administrative Region")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Micronesia.*$)', "Micronesia (Federated States of)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Moldova.*$)', "Republic of Moldova (the)")
df['Country Name'] = df['Country Name'].replace("Morroco",'Morocco')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Nepal.*$)', "Nepal")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*New Ze.*$)', "New Zealand")
df['Country Name'] = df['Country Name'].replace("Niger",'Niger (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macedonia.*$)', "North Macedonia")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*New Ze.*$)', "New Zealand")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Palestin.*$)', "State of Palestine (the)")
df['Country Name'] = df['Country Name'].replace("West Bank and Gaza",'State of Palestine (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Panama.*$)', "Panama")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Philippines.*$)', "Philippines (the)")
df['Country Name'] = df['Country Name'].replace("Republic of the Congo",'Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Myanmar.*$)', "Myanmar")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Puerto Rico.*$)', "Puerto Rico")
df['Country Name'] = df['Country Name'].replace("Russia",'Russian Federation (the)')
df['Country Name'] = df['Country Name'].replace("Russian Federation",'Russian Federation (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Slovak.*$)', "Slovakia")
df['Country Name'] = df['Country Name'].str.replace(r'\bSudan\b', 'Sudan (the)')
df['Country Name'] = df['Country Name'].str.replace(r'\bSudan (the)\b', 'Sudan (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*South Sudan.*$)', "South Sudan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Syria.*$)', "Syrian Arab Republic (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*São Tomé.*$)', "Sao Tome and Principe")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Taiwan.*$)', "Taiwan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Taipei.*$)', "Taiwan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Tanzania.*$)', "United Republic of Tanzania (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Netherlands.*$)', "Netherlands (the)")
df['Country Name'] = df['Country Name'].replace("UAE",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace("U.A.E",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace("United Arab Emirates",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace('United Kingdom','United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace('UK','United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace("Great Britain",'United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace("United Kingdom of Great Britain and Northern Ireland",'United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace('Vietnam','Viet Nam')
df['Country Name'] = df['Country Name'].replace('United States','United States of America (the)')
df['Country Name'] = df['Country Name'].replace('USA','United States of America (the)')
df['Country Name'] = df['Country Name'].replace('United States of America','United States of America (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Virgin Islands.*$)', "United States Virgin Islands")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vatican.*$)', "Vatican")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Venezuela.*$)', "Venezuela (Bolivarian Republic of)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Yemen.*$)', "Yemen")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Arab world.*$)', "Arab World")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*World.*$)', "World")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kitts and Nevis.*$)', "Saint Kitts and Nevis")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Lucia.*$)', "Saint Lucia")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Martin (French Part).*$)', "Saint Martin (French Part)")
df['Country Name'] = df['Country Name'].replace('Sint Maarten','Saint Martin')
df['Country Name'] = df['Country Name'].replace('St. Martin (French part)','Saint Martin (French Part)')
df['Country Name'] = df['Country Name'].replace('Sint Maarten (Dutch part)','Saint Martin (Dutch Part)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vincent and the Grenadines.*$)', "Saint Vincent and the Grenadines")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Verde.*$)', "Cabo Verde")
df['Country Name'] = df['Country Name'].replace('Congo, Democratic Republic','Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace('Congo, Rep.','Congo (the)')
df['Country Name'] = df['Country Name'].replace('Republic of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].replace('Congo (Rep.)','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Egypt.*$)', "Egypt")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Korea, D.*$)', "Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Tobago.*$)', "Trinidad and Tobago")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Timor-Leste.*$)', "Timor-Leste")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Emirates.*$)', "United Arab Emirates (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Papua.*$)', "Papua New Guinea")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bissau.*$)', "Guinea-Bissau")
df['Country Name'] = df['Country Name'].replace('Eq. Guinea','Equatorial Guinea')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Burma.*$)', "Myanmar")
df['Country Name'] = df['Country Name'].replace('C.A. Republic','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace('Ant.& Barb.','Antigua and Barbuda')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bosnia.*$)', "Bosnia and Herzegovina")
df['Country Name'] = df['Country Name'].replace('Domin. Rep.','Dominican Republic (the)')
df['Country Name'] = df['Country Name'].replace('Dominica (Commonwealth of)','Dominica')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*European Union.*$)', "European Union")
df['Country Name'] = df['Country Name'].replace('R. of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Principe.*$)', "Sao Tome and Principe")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Solomon.*$)', "Solomon Islands")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vincent.*$)', "Saint Vincent and the Grenadines")
df['Country Name'] = df['Country Name'].replace('Curacao','Curaçao')
df['Country Name'] = df['Country Name'].replace('Reunion','Réunion')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kosovo.*$)', "Kosovo (UNSCR 1244)")

df=df.merge(pop,how='outer',on='Country Name')
df

<ipython-input-155-709cf5e28ea8>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bahamas.*$)', 'Bahamas (the)')
<ipython-input-155-709cf5e28ea8>:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r"(^.*Côte d'Ivoire.*$)", "Côte d'Ivoire")
<ipython-input-155-709cf5e28ea8>:35: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Czech.*$)', 'Czechia')
<ipython-input-155-709cf5e28ea8>:36: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Dominican Re.*$)', 'Dominican Republic (the)')
<ipython-input-155-709cf5e28ea8>:37: FutureWarning: The default value of regex will change from Tru

Country Name    Year  \
0              Australia  2019.0   
1                Austria  2019.0   
2                Belgium  2019.0   
3                 Canada  2019.0   
4                Czechia  2019.0   
..                   ...     ...   
261                Samoa     NaN   
262  Kosovo (UNSCR 1244)     NaN   
263                Yemen     NaN   
264               Zambia     NaN   
265             Zimbabwe     NaN   

                                             Indicator     data_col  \
0    Amount invested into startups yearly from priv...   487.310802   
1    Amount invested into startups yearly from priv...    90.416881   
2    Amount invested into startups yearly from priv...   395.828165   
3    Amount invested into startups yearly from priv...  3286.872933   
4    Amount invested into startups yearly from priv...    27.204683   
..                                                 ...          ...   
261                                                NaN          NaN   
262                                                NaN          NaN   
263                                                NaN          NaN   
264                                                NaN          NaN   
265                                                NaN          NaN   

    higher_is_better           Sub-Pillar        1960        1961        1962  \
0               True  Startup Environment  10276477.0  10483000.0  10742000.0   
1               True  Startup Environment   7047539.0   7086299.0   7129864.0   
2               True  Startup Environment   9153489.0   9183948.0   9220578.0   
3               True  Startup Environment  17909009.0  18271000.0  18614000.0   
4               True  Startup Environment   9602006.0   9586651.0   9624660.0   
..               ...                  ...         ...         ...         ...   
261              NaN                  NaN    108627.0    112112.0    115768.0   
262              NaN                  NaN    947000.0    966000.0    994000.0   
263              NaN                  NaN   5315351.0   5393034.0   5473671.0   
264              NaN                  NaN   3070780.0   3164330.0   3260645.0   
265              NaN                  NaN   3776679.0   3905038.0   4039209.0   

           1963  ...        2014        2015        2016        2017  \
0    10950000.0  ...  23475686.0  23815995.0  24190907.0  24601860.0   
1     7175811.0  ...   8546356.0   8642699.0   8736668.0   8797566.0   
2     9289770.0  ...  11209057.0  11274196.0  11331422.0  11375158.0   
3    18964000.0  ...  35437435.0  35702908.0  36109487.0  36545295.0   
4     9670685.0  ...  10525347.0  10546059.0  10566332.0  10594438.0   
..          ...  ...         ...         ...         ...         ...   
261    119552.0  ...    192220.0    193510.0    194540.0    195358.0   
262   1022000.0  ...   1812771.0   1788196.0   1777557.0   1791003.0   
263   5556767.0  ...  25823488.0  26497881.0  27168210.0  27834811.0   
264   3360099.0  ...  15399793.0  15879370.0  16363449.0  16853608.0   
265   4178726.0  ...  13586710.0  13814642.0  14030338.0  14236599.0   

           2018        2019        2020  Country Code     Indicator Name  \
0    24982688.0  25365745.0  25687041.0           AUS  Population, total   
1     8840521.0   8879920.0   8917205.0           AUT  Population, total   
2    11427054.0  11488980.0  11555997.0           BEL  Population, total   
3    37065178.0  37593384.0  38005238.0           CAN  Population, total   
4    10629928.0  10671870.0  10698896.0           CZE  Population, total   
..          ...         ...         ...           ...                ...   
261    196128.0    197093.0    198410.0           WSM  Population, total   
262   1797085.0   1788878.0   1775378.0           XKX  Population, total   
263  28498683.0  29161922.0  29825968.0           YEM  Population, total   
264  17351714.0  17861034.0  18383956.0           ZMB  Population, total   
265  14438812.0  14645473.0  14862927.0           ZW

In [156]:
df['new_data_col'] = (df['data_col']*1000000)/(df['2019'])
df=df[df['Indicator'].notna()]
df['new_data_col'] = df['new_data_col'].astype(np.float64)
df=df[['Country Name','Year','Indicator','new_data_col','higher_is_better','Sub-Pillar']]
min_rank = df['new_data_col'].min()
max_rank = df['new_data_col'].max()

<ipython-input-156-19aa0e3619d9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_data_col'] = df['new_data_col'].astype(np.float64)


In [157]:
first = df['new_data_col'].quantile(1/5)
second = df['new_data_col'].quantile(2/5)
third = df['new_data_col'].quantile(3/5)
fourth = df['new_data_col'].quantile(4/5)
fifth = df['new_data_col'].quantile(5/5)
data = {'Min':min_rank,'Max':max_rank,'1':first,'2':second,'3':third,'4':fourth,'5':fifth}
sextile = pd.DataFrame(data, index=[0])
sextile

Min         Max         1          2          3          4           5
0  0.566176  413.147474  3.219303  14.089509  29.409678  44.330586  413.147474

In [158]:
df.rename(columns={'new_data_col':'data_col'}, inplace=True)
min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

df.loc[df['data_col'] <= first, 'new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_I(row, old_min=min_rank,old_max=first))
df.loc[(df['data_col'] > first) & (df['data_col'] <= second), 'new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_II(row, old_min=first,old_max=second))
df.loc[(df['data_col'] > second) & (df['data_col'] <= third), 'new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_III(row, old_min=second,old_max=third))
df.loc[(df['data_col'] > third) & (df['data_col'] <= fourth), 'new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_IV(row, old_min=third,old_max=fourth))
df.loc[df['data_col'] > fourth, 'new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_V(row, old_min=fourth,old_max=max_rank))
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

Country Name    Year  \
0                                           Australia  2019.0   
1                                             Austria  2019.0   
2                                             Belgium  2019.0   
3                                              Canada  2019.0   
4                                             Czechia  2019.0   
5                                             Denmark  2019.0   
6                                             Finland  2019.0   
7                                              France  2019.0   
8                                             Germany  2019.0   
9                                              Greece  2019.0   
10                                            Hungary  2019.0   
11                                            Ireland  2019.0   
12                                              Italy  2019.0   
13                                              Japan  2019.0   
14                            Republic of Korea (the)  2019.0   
15                                         Luxembourg  2019.0   
16                                  Netherlands (the)  2019.0   
17                                        New Zealand  2019.0   
18                                             Norway  2019.0   
19                                             Poland  2019.0   
20                                           Portugal  2019.0   
21                                           Slovakia  2019.0   
22                                              Spain  2019.0   
23                                             Sweden  2019.0   
24                                        Switzerland  2019.0   
25  United Kingdom of Great Britain and Northern I...  2019.0   
26                     United States of America (the)  2019.0   
27                                            Estonia  2019.0   
28                                             Latvia  2019.0   
29                                          Lithuania  2019.0   
30                                            Romania  2019.0   
31                           Russian Federation (the)  2019.0   
32                                       South Africa  2019.0   
33                                           Bulgaria  2019.0   

                                            Indicator    data_col  \
0   Amount invested into startups yearly from priv...   19.211374   
1   Amount invested into startups yearly from priv...   10.182173   
2   Amount invested into startups yearly from priv...   34.452855   
3   Amount invested into startups yearly from priv...   87.432218   
4   Amount invested into startups yearly from priv...    2.549195   
5   Amount invested into startups yearly from priv...   60.614360   
6   Amount invested into startups yearly from priv...   58.237866   
7   Amount invested into startups yearly from priv...   32.192997   
8   Amount invested into startups yearly from priv...   28.638680   
9   Amount invested into startups yearly from priv...    2.523680   
10  Amount invested into startups yearly from priv...   15.849573   
11  Amount invested into startups yearly from priv...   37.478934   
12  Amount invested into startups yearly from priv...    4.371661   
13  Amount invested into startups yearly from priv...   19.819679   
14  Amount invested into startups yearly from priv...   52.378389   
15  Amount invested into startups yearly from priv...   27.915468   
16  Amount invested into startups yearly from priv...   40.469569   
17  Amount invested into startups yearly from priv...   14.845286   
18  Amount invested into startups yearly from priv...   29.602427   
19  Amount invested into startups yearly from priv...    2.975933   
20  Amount invested into startups yearly from priv...    5.005935   
21  Amount invested into startups yearly from priv...    4.509806   
22  Amount invested into startups yearly from priv...   13.900565   
23  Amount invested into startups yearly from priv...   39.164971   
24  Amount invested into startups yearly

In [159]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(bf), index=False)
df.to_csv('../non-index/business_{}_scores.csv'.format(bf),index = False)

In [160]:
### 17. Regulatory Quality

In [161]:
indicator = indicators[16]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))
df.head(16)

Regulatory Quality
global_innovation_dataset


ISO2  Economy       IndCode IndNum                               IndName  \
0   ALB  Albania        Inputs     .1            Innovation Input Sub-index   
1   ALB  Albania       Outputs     .2           Innovation Output Sub-index   
2   ALB  Albania         Index      0               Global Innovation Index   
3   ALB  Albania            P1      1                          Institutions   
4   ALB  Albania          SP11   1.1.                 Political environment   
5   ALB  Albania       PolStab  1.1.1  Political and operational stability*   
6   ALB  Albania        GovEff  1.1.2             Government effectiveness*   
7   ALB  Albania          SP12   1.2.                Regulatory environment   
8   ALB  Albania        RegQua  1.2.1                   Regulatory quality*   
9   ALB  Albania        RuleOL  1.2.2                          Rule of law*   
10  ALB  Albania      CostRedu  1.2.3           Cost of redudancy dismissal   
11  ALB  Albania          SP13   1.3.                  Business environment   
12  ALB  Albania  EaseStartBus  1.3.1          Ease of starting a business*   
13  ALB  Albania     EaseInsol  1.3.2         Ease of resolving insolvency*   
14  ALB  Albania            P2      2            Human capital and research   
15  ALB  Albania          SP21   2.1.                             Education   

    Rank      Score      Value  MinDatCov  Outdated OverallStreWeak  \
0   71.0  39.940929        NaN        NaN       NaN             NaN   
1   92.0  16.109792        NaN        NaN       NaN             NaN   
2   84.0  28.025361        NaN        NaN       NaN             NaN   
3   60.0  64.919240        NaN        0.0       NaN             NaN   
4   71.0  56.071183        NaN        0.0       NaN             NaN   
5   60.0  69.642859   2.100000        NaN       0.0             NaN   
6   76.0  49.285344  -0.061331        NaN       0.0             NaN   
7   82.0  58.941532        NaN        0.0       NaN             NaN   
8   58.0  50.670701   0.274380        NaN       0.0             NaN   
9   85.0  35.889077  -0.411179        NaN       0.0             NaN   
10  90.0  74.603175  20.825397        NaN       0.0             NaN   
11  34.0  79.745005        NaN        0.0       NaN               S   
12  47.0  91.759090  91.759090        NaN       0.0             NaN   
13  36.0  67.730920  67.730920        NaN       0.0               S   
14  90.0  22.673981        NaN        0.0       NaN             NaN   
15  95.0  39.763678        NaN        0.0       NaN             NaN   

   IncomeStreWeak  DataYear  
0             NaN       NaN  
1             NaN       NaN  
2             NaN       NaN  
3             NaN       NaN  
4             NaN       NaN  
5             NaN    2020.0  
6             NaN    2019.0  
7             NaN       NaN  
8             NaN    2019.0  
9             NaN    2019.0  
10            NaN    2019.0  
11              S       NaN  
12            NaN    2019.0  
13            NaN    2019.0  
14            NaN       NaN  
15            NaN       NaN

In [162]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Startup Environment


In [163]:
df = df[(df.IndName == 'Regulatory quality*')]
df

ISO2     Economy IndCode IndNum              IndName   Rank      Score  \
8      ALB     Albania  RegQua  1.2.1  Regulatory quality*   58.0  50.670701   
120    DZA     Algeria  RegQua  1.2.1  Regulatory quality*  129.0   9.425411   
232    AGO      Angola  RegQua  1.2.1  Regulatory quality*  124.0  20.130634   
344    ARG   Argentina  RegQua  1.2.1  Regulatory quality*  103.0  30.616559   
456    ARM     Armenia  RegQua  1.2.1  Regulatory quality*   59.0  50.020701   
...    ...         ...     ...    ...                  ...    ...        ...   
14232  UZB  Uzbekistan  RegQua  1.2.1  Regulatory quality*  126.0  17.531597   
14344  VNM    Viet Nam  RegQua  1.2.1  Regulatory quality*   93.0  36.639718   
14456  YEM       Yemen  RegQua  1.2.1  Regulatory quality*  132.0   0.000000   
14568  ZMB      Zambia  RegQua  1.2.1  Regulatory quality*  105.0  29.008412   
14680  ZWE    Zimbabwe  RegQua  1.2.1  Regulatory quality*  131.0   5.247431   

          Value  MinDatCov  Outdated OverallStreWeak IncomeStreWeak  DataYear  
8      0.274380        NaN       0.0             NaN            NaN    2019.0  
120   -1.303379        NaN       0.0             NaN              W    2019.0  
232   -0.893871        NaN       0.0             NaN            NaN    2019.0  
344   -0.492753        NaN       0.0             NaN              W    2019.0  
456    0.249515        NaN       0.0             NaN            NaN    2019.0  
...         ...        ...       ...             ...            ...       ...  
14232 -0.993293        NaN       0.0               W              W    2019.0  
14344 -0.262348        NaN       0.0             NaN            NaN    2019.0  
14456 -1.663930        NaN       0.0               W              W    2019.0  
14568 -0.554269        NaN       0.0             NaN            NaN    2019.0  
14680 -1.463199        NaN       0.0               W              W    2019.0  

[132 rows x 13 columns]

In [164]:
df['higher_is_better'] = True
df['Year'] = df['DataYear']
df['Indicator'] = indicator
df['data_col'] = df['Score']
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['Economy']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [165]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

Country Name    Year           Indicator   data_col  new_rank_score  \
8          Albania  2019.0  Regulatory Quality  50.670701        3.528468   
120        Algeria  2019.0  Regulatory Quality   9.425411        1.470328   
232         Angola  2019.0  Regulatory Quality  20.130634        2.004519   
344      Argentina  2019.0  Regulatory Quality  30.616559        2.527766   
456        Armenia  2019.0  Regulatory Quality  50.020701        3.496033   
...            ...     ...                 ...        ...             ...   
14232   Uzbekistan  2019.0  Regulatory Quality  17.531597        1.874827   
14344     Viet Nam  2019.0  Regulatory Quality  36.639718        2.828322   
14456        Yemen  2019.0  Regulatory Quality   0.000000        1.000000   
14568       Zambia  2019.0  Regulatory Quality  29.008412        2.447520   
14680     Zimbabwe  2019.0  Regulatory Quality   5.247431        1.261847   

       higher_is_better           Sub-Pillar  
8                  True  Startup Environment  
120                True  Startup Environment  
232                True  Startup Environment  
344                True  Startup Environment  
456                True  Startup Environment  
...                 ...                  ...  
14232              True  Startup Environment  
14344              True  Startup Environment  
14456              True  Startup Environment  
14568              True  Startup Environment  
14680              True  Startup Environment  

[132 rows x 7 columns]

In [166]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

In [167]:
### 18. Ease of Getting Credit

In [168]:
indicator = indicators[17]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))
df.head(16)

Ease of Getting Credit
global_innovation_dataset


ISO2  Economy       IndCode IndNum                               IndName  \
0   ALB  Albania        Inputs     .1            Innovation Input Sub-index   
1   ALB  Albania       Outputs     .2           Innovation Output Sub-index   
2   ALB  Albania         Index      0               Global Innovation Index   
3   ALB  Albania            P1      1                          Institutions   
4   ALB  Albania          SP11   1.1.                 Political environment   
5   ALB  Albania       PolStab  1.1.1  Political and operational stability*   
6   ALB  Albania        GovEff  1.1.2             Government effectiveness*   
7   ALB  Albania          SP12   1.2.                Regulatory environment   
8   ALB  Albania        RegQua  1.2.1                   Regulatory quality*   
9   ALB  Albania        RuleOL  1.2.2                          Rule of law*   
10  ALB  Albania      CostRedu  1.2.3           Cost of redudancy dismissal   
11  ALB  Albania          SP13   1.3.                  Business environment   
12  ALB  Albania  EaseStartBus  1.3.1          Ease of starting a business*   
13  ALB  Albania     EaseInsol  1.3.2         Ease of resolving insolvency*   
14  ALB  Albania            P2      2            Human capital and research   
15  ALB  Albania          SP21   2.1.                             Education   

    Rank      Score      Value  MinDatCov  Outdated OverallStreWeak  \
0   71.0  39.940929        NaN        NaN       NaN             NaN   
1   92.0  16.109792        NaN        NaN       NaN             NaN   
2   84.0  28.025361        NaN        NaN       NaN             NaN   
3   60.0  64.919240        NaN        0.0       NaN             NaN   
4   71.0  56.071183        NaN        0.0       NaN             NaN   
5   60.0  69.642859   2.100000        NaN       0.0             NaN   
6   76.0  49.285344  -0.061331        NaN       0.0             NaN   
7   82.0  58.941532        NaN        0.0       NaN             NaN   
8   58.0  50.670701   0.274380        NaN       0.0             NaN   
9   85.0  35.889077  -0.411179        NaN       0.0             NaN   
10  90.0  74.603175  20.825397        NaN       0.0             NaN   
11  34.0  79.745005        NaN        0.0       NaN               S   
12  47.0  91.759090  91.759090        NaN       0.0             NaN   
13  36.0  67.730920  67.730920        NaN       0.0               S   
14  90.0  22.673981        NaN        0.0       NaN             NaN   
15  95.0  39.763678        NaN        0.0       NaN             NaN   

   IncomeStreWeak  DataYear  
0             NaN       NaN  
1             NaN       NaN  
2             NaN       NaN  
3             NaN       NaN  
4             NaN       NaN  
5             NaN    2020.0  
6             NaN    2019.0  
7             NaN       NaN  
8             NaN    2019.0  
9             NaN    2019.0  
10            NaN    2019.0  
11              S       NaN  
12            NaN    2019.0  
13            NaN    2019.0  
14            NaN       NaN  
15            NaN       NaN

In [169]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Startup Environment


In [170]:
df = df[(df.IndName == 'Ease of getting credit*')]
df

ISO2     Economy   IndCode IndNum                  IndName   Rank  \
46     ALB     Albania  EaseCred  4.1.1  Ease of getting credit*   44.0   
158    DZA     Algeria  EaseCred  4.1.1  Ease of getting credit*  129.0   
270    AGO      Angola  EaseCred  4.1.1  Ease of getting credit*  131.0   
382    ARG   Argentina  EaseCred  4.1.1  Ease of getting credit*   94.0   
494    ARM     Armenia  EaseCred  4.1.1  Ease of getting credit*   44.0   
...    ...         ...       ...    ...                      ...    ...   
14270  UZB  Uzbekistan  EaseCred  4.1.1  Ease of getting credit*   61.0   
14382  VNM    Viet Nam  EaseCred  4.1.1  Ease of getting credit*   23.0   
14494  YEM       Yemen  EaseCred  4.1.1  Ease of getting credit*  132.0   
14606  ZMB      Zambia  EaseCred  4.1.1  Ease of getting credit*    4.0   
14718  ZWE    Zimbabwe  EaseCred  4.1.1  Ease of getting credit*   61.0   

       Score  Value  MinDatCov  Outdated OverallStreWeak IncomeStreWeak  \
46      70.0   70.0        NaN       0.0             NaN            NaN   
158     10.0   10.0        NaN       0.0               W              W   
270      5.0    5.0        NaN       0.0               W              W   
382     50.0   50.0        NaN       0.0             NaN              W   
494     70.0   70.0        NaN       0.0             NaN            NaN   
...      ...    ...        ...       ...             ...            ...   
14270   65.0   65.0        NaN       0.0             NaN            NaN   
14382   80.0   80.0        NaN       0.0             NaN            NaN   
14494    0.0    0.0        NaN       0.0               W              W   
14606   95.0   95.0        NaN       0.0               S              S   
14718   65.0   65.0        NaN       0.0             NaN            NaN   

       DataYear  
46       2019.0  
158      2019.0  
270      2019.0  
382      2019.0  
494      2019.0  
...         ...  
14270    2019.0  
14382    2019.0  
14494    2019.0  
14606    2019.0  
14718    2019.0  

[132 rows x 13 columns]

In [171]:
df['higher_is_better'] = True
df['Year'] = df['DataYear']
df['Indicator'] = indicator
df['data_col'] = df['Score']
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['Economy']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [172]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

Country Name    Year               Indicator  data_col  new_rank_score  \
46         Albania  2019.0  Ease of Getting Credit      70.0          4.4930   
158        Algeria  2019.0  Ease of Getting Credit      10.0          1.4990   
270         Angola  2019.0  Ease of Getting Credit       5.0          1.2495   
382      Argentina  2019.0  Ease of Getting Credit      50.0          3.4950   
494        Armenia  2019.0  Ease of Getting Credit      70.0          4.4930   
...            ...     ...                     ...       ...             ...   
14270   Uzbekistan  2019.0  Ease of Getting Credit      65.0          4.2435   
14382     Viet Nam  2019.0  Ease of Getting Credit      80.0          4.9920   
14494        Yemen  2019.0  Ease of Getting Credit       0.0          1.0000   
14606       Zambia  2019.0  Ease of Getting Credit      95.0          5.7405   
14718     Zimbabwe  2019.0  Ease of Getting Credit      65.0          4.2435   

       higher_is_better           Sub-Pillar  
46                 True  Startup Environment  
158                True  Startup Environment  
270                True  Startup Environment  
382                True  Startup Environment  
494                True  Startup Environment  
...                 ...                  ...  
14270              True  Startup Environment  
14382              True  Startup Environment  
14494              True  Startup Environment  
14606              True  Startup Environment  
14718              True  Startup Environment  

[132 rows x 7 columns]

In [173]:
df.to_csv('../indicator_scores/business_{}_scores.csv'.format(indicator), index=False)

In [174]:
### 19. Ease of Getting Credit

In [175]:
indicator = indicators[18]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))
df.head(16)

Global Resilience Index
global_resilience_index


Country  Global Resilience Score  Unnamed: 2
0             Albania                     34.6         NaN
1             Algeria                     31.2         NaN
2           Argentina                     53.7         NaN
3             Armenia                     46.9         NaN
4           Australia                     90.3         NaN
5             Austria                     94.2         NaN
6          Azerbaijan                     40.6         NaN
7             Bahrain                     60.7         NaN
8          Bangladesh                     29.5         NaN
9             Belgium                     90.1         NaN
10              Benin                     31.5         NaN
11            Bolivia                     27.3         NaN
12             Bosnia                     47.9         NaN
13           Botswana                     56.9         NaN
14             Brazil                     52.1         NaN
15  Brunei Darussalam                     65.6         NaN

In [176]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Startup Environment


In [177]:
df['higher_is_better'] = True
df['Year'] = 2021
df['Indicator'] = indicator
df['data_col'] = df['Global Resilience Score']
df['Sub-Pillar'] = subpillar
df['Country Name'] = df['Country']

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [178]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

Country Name  Year                Indicator  data_col  new_rank_score  \
0        Albania  2021  Global Resilience Index      34.6         2.72654   
1        Algeria  2021  Global Resilience Index      31.2         2.55688   
2      Argentina  2021  Global Resilience Index      53.7         3.67963   
3        Armenia  2021  Global Resilience Index      46.9         3.34031   
4      Australia  2021  Global Resilience Index      90.3         5.50597   
..           ...   ...                      ...       ...             ...   
121      Uruguay  2021  Global Resilience Index      62.3         4.10877   
122    Venezuela  2021  Global Resilience Index       1.5         1.07485   
123      Vietnam  2021  Global Resilience Index      37.9         2.89121   
124       Zambia  2021  Global Resilience Index      32.5         2.62175   
125     Zimbabwe  2021  Global Resilience Index      34.0         2.69660   

     higher_is_better           Sub-Pillar  
0                True  Startup Environment  
1                True  Startup Environment  
2                True  Startup Environment  
3                True  Startup Environment  
4                True  Startup Environment  
..                ...                  ...  
121              True  Startup Environment  
122              True  Startup Environment  
123              True  Startup Environment  
124              True  Startup Environment  
125              True  Startup Environment  

[126 rows x 7 columns]

### Score Aggregating

In [179]:
import os


In [180]:
# get list of files in scores folder
scores = os.listdir('../indicator_scores/')
scores = [s for s in scores if s.startswith('business')]

In [181]:
scores

['business_Cloud Services (Spend, IT Forecast Data)_scores.csv',
 'business_Doing Business Index_scores.csv',
 'business_Ease of finding skilled employees_scores.csv',
 'business_Ease of Getting Credit_scores.csv',
 'business_ICT task-intensive jobs as a percentage of total employment_scores.csv',
 'business_Networking Services (Spend, IT Forecast Data)_scores.csv',
 'business_Regulatory Quality_scores.csv',
 'business_Share of business with internet_scores.csv',
 'business_Share of businesses with broadband_scores.csv',
 'business_Share of businesses with online presence_scores.csv',
 'business_Size of digital economy (% of transactions)_scores.csv',
 'business_Size of gig economy (% of GDP)_scores.csv',
 'business_start_up_investment_scores.csv',
 'business_Strength of Legal Rights_scores.csv',
 'business_Time to Start a Business_scores.csv',
 'business_UNCTAD Business-to-Consumer (B2C) E-commerce Index_scores.csv',
 'business_Venture Capital Availability_scores.csv']

In [182]:
# create a dataframe that concatenates all these file into one table
df = pd.concat([pd.read_csv('../indicator_scores/{}'.format(s)) for s in scores])    

In [183]:
df

Country Name    Year                                 Indicator  \
0           Germany  2018.0  Cloud Services (Spend, IT Forecast Data)   
1             Japan  2018.0  Cloud Services (Spend, IT Forecast Data)   
2     United States  2018.0  Cloud Services (Spend, IT Forecast Data)   
3    United Kingdom  2018.0  Cloud Services (Spend, IT Forecast Data)   
4         Australia  2018.0  Cloud Services (Spend, IT Forecast Data)   
..              ...     ...                                       ...   
147         Vietnam  2019.0              Venture Capital Availability   
148     Yemen, Rep.  2019.0              Venture Capital Availability   
149    South Africa  2019.0              Venture Capital Availability   
150          Zambia  2019.0              Venture Capital Availability   
151        Zimbabwe  2019.0              Venture Capital Availability   

      data_col  new_rank_score  higher_is_better            Sub-Pillar  
0    18.200000        1.908180              True   Technology Adoption  
1    20.300000        2.012970              True   Technology Adoption  
2    18.000000        1.898200              True   Technology Adoption  
3    19.800000        1.988020              True   Technology Adoption  
4    16.100000        1.803390              True   Technology Adoption  
..         ...             ...               ...                   ...  
147   3.267500        2.885804              True  Financing Incentives  
148   2.163688        1.967801              True  Financing Incentives  
149   3.051583        2.706233              True  Financing Incentives  
150   1.803466        1.668216              True  Financing Incentives  
151   2.058023        1.879922              True  Financing Incentives  

[2101 rows x 7 columns]

In [184]:
# Data cleaning
df['new_rank_score'] = df['new_rank_score'].fillna(0)
df.sort_values(by=['Country Name'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country Name      2101 non-null   object 
 1   Year              2101 non-null   float64
 2   Indicator         2101 non-null   object 
 3   data_col          2019 non-null   float64
 4   new_rank_score    2101 non-null   float64
 5   higher_is_better  2101 non-null   bool   
 6   Sub-Pillar        2101 non-null   object 
dtypes: bool(1), float64(3), object(3)
memory usage: 100.7+ KB


In [186]:
df

Country Name    Year  \
0                     Afghanistan  2019.0   
1                     Afghanistan  2020.0   
2                     Afghanistan  2019.0   
3                     Afghanistan  2019.0   
4     Africa Eastern and Southern  2019.0   
...                           ...     ...   
2096                     Zimbabwe  2019.0   
2097                     Zimbabwe  2019.0   
2098                     Zimbabwe  2019.0   
2099                     Zimbabwe  2019.0   
2100                    Zimbabwe   2020.0   

                                              Indicator   data_col  \
0     Strength of legal rights index (0=weak to 12=s...  10.000000   
1     UNCTAD Business-to-Consumer (B2C) E-commerce I...  17.100000   
2                                  Doing Business Index  47.770000   
3              Time required to start a business (days)   8.500000   
4     Strength of legal rights index (0=weak to 12=s...   4.538462   
...                                                 ...        ...   
2096                     Size of gig economy (% of GDP)   0.000000   
2097        Size of digital economy (% of transactions)  21.820000   
2098                             Ease of Getting Credit  65.000000   
2099                       Venture Capital Availability   2.058023   
2100      Networking Services (Spend, IT Forecast Data)  25.780000   

      new_rank_score  higher_is_better            Sub-Pillar  
0           5.158333              True   Startup Environment  
1           1.853290              True   Technology Adoption  
2           3.383723             False   Startup Environment  
3           4.660000             False   Startup Environment  
4           2.887244              True   Startup Environment  
...              ...               ...                   ...  
2096        1.000000              True   Technology Adoption  
2097        2.088818              True   Technology Adoption  
2098        4.243500              True   Startup Environment  
2099        1.879922              True  Financing Incentives  
2100        2.286422              True   Technology Adoption  

[2101 rows x 7 columns]

In [187]:
df.head(15)

Country Name    Year  \
0                   Afghanistan  2019.0   
1                   Afghanistan  2020.0   
2                   Afghanistan  2019.0   
3                   Afghanistan  2019.0   
4   Africa Eastern and Southern  2019.0   
5   Africa Eastern and Southern  2019.0   
6    Africa Western and Central  2019.0   
7    Africa Western and Central  2019.0   
8                       Albania  2019.0   
9                       Albania  2019.0   
10                      Albania  2019.0   
11                      Albania  2019.0   
12                      Albania  2020.0   
13                      Albania  2019.0   
14                      Albania  2018.0   

                                            Indicator   data_col  \
0   Strength of legal rights index (0=weak to 12=s...  10.000000   
1   UNCTAD Business-to-Consumer (B2C) E-commerce I...  17.100000   
2                                Doing Business Index  47.770000   
3            Time required to start a business (days)   8.500000   
4   Strength of legal rights index (0=weak to 12=s...   4.538462   
5            Time required to start a business (days)  26.826923   
6   Strength of legal rights index (0=weak to 12=s...   5.863636   
7            Time required to start a business (days)  15.259091   
8                      Size of gig economy (% of GDP)  28.160000   
9   Strength of legal rights index (0=weak to 12=s...   8.000000   
10        Size of digital economy (% of transactions)   6.690000   
11                               Doing Business Index  69.510000   
12  UNCTAD Business-to-Consumer (B2C) E-commerce I...  49.500000   
13           Time required to start a business (days)   4.500000   
14           Share of businesses with online presence  59.800000   

    new_rank_score  higher_is_better           Sub-Pillar  
0         5.158333              True  Startup Environment  
1         1.853290              True  Technology Adoption  
2         3.383723             False  Startup Environment  
3         4.660000             False  Startup Environment  
4         2.887244              True  Startup Environment  
5         1.983835             False  Startup Environment  
6         3.438295              True  Startup Environment  
7         3.325446             False  Startup Environment  
8         2.405184              True  Technology Adoption  
9         4.326667              True  Startup Environment  
10        1.333831              True  Technology Adoption  
11        4.468549             False  Startup Environment  
12        3.470050              True  Technology Adoption  
13        5.380769             False  Startup Environment  
14        3.984020              True  Technology Adoption

In [188]:
df.describe()

Year     data_col  new_rank_score
count  2101.000000  2019.000000     2101.000000
mean   2018.971442    35.477754        3.393896
std       1.023134    30.413121        1.349983
min    2010.000000     0.000000        0.000000
25%    2019.000000     6.000000        2.460573
50%    2019.000000    30.000000        3.495000
75%    2019.000000    60.000000        4.451417
max    2020.000000   413.147474        5.990000

In [189]:
df['Country Name'] = df['Country Name'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country Name      2101 non-null   object 
 1   Year              2101 non-null   float64
 2   Indicator         2101 non-null   object 
 3   data_col          2019 non-null   float64
 4   new_rank_score    2101 non-null   float64
 5   higher_is_better  2101 non-null   bool   
 6   Sub-Pillar        2101 non-null   object 
dtypes: bool(1), float64(3), object(3)
memory usage: 100.7+ KB


In [190]:
# checking country names
sorted(df['Country Name'].unique().tolist())

['Afghanistan',
 'Africa Eastern and Southern',
 'Africa Western and Central',
 'Albania',
 'Albania ',
 'Algeria',
 'Algeria ',
 'American Samoa',
 'Andorra',
 'Angola',
 'Angola ',
 'Antigua and Barbuda',
 'Arab World',
 'Argentina',
 'Argentina ',
 'Armenia',
 'Armenia ',
 'Aruba',
 'Australia',
 'Australia ',
 'Austria',
 'Austria ',
 'Azerbaijan',
 'Azerbaijan ',
 'B:',
 'Bahamas, The',
 'Bahrain',
 'Bahrain ',
 'Bangladesh',
 'Bangladesh ',
 'Bangladesh Chittagong',
 'Bangladesh Dhaka',
 'Barbados',
 'Belarus',
 'Belarus ',
 'Belgium',
 'Belgium ',
 'Belize',
 'Benin',
 'Benin ',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bolivia ',
 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina',
 'Bosnia and Herzegovina ',
 'Botswana',
 'Botswana ',
 'Brazil',
 'Brazil ',
 'Brazil Rio de Janeiro',
 'Brazil São Paulo',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Bulgaria ',
 'Burkina Faso',
 'Burkina Faso ',
 'Burundi',
 'Burundi ',
 'Cabo Verde',
 'Cabo Verde ',
 'C

In [191]:
# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()
df['Country Name'] = df['Country Name'].str.strip('**')
df['Country Name'] = df['Country Name'].str.strip('*')

In [192]:
# checking country names
sorted(df['Country Name'].unique().tolist())

['Afghanistan',
 'Africa Eastern and Southern',
 'Africa Western and Central',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Arab World',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'B:',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Bangladesh Chittagong',
 'Bangladesh Dhaka',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brazil Rio de Janeiro',
 'Brazil São Paulo',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Caribbean small states',
 'Cayman Islands',
 'Central African Republic',
 'Central Europe and the Baltics',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 'China Beijing',
 'China Shanghai',
 'China, Hong Kong SAR',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo, Dem. 

In [193]:
# average indicator scores per country
agg_df = df.groupby(['Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [194]:
agg_df.columns = ['agg_score', 'count_source' ]

In [195]:
max_number_sources = agg_df.describe()['count_source']['max']

In [196]:
agg_df['agg_score_wt'] = agg_df['agg_score']*(agg_df['count_source']/max_number_sources)

In [197]:
agg_df.sort_values(by='agg_score', ascending=False, inplace=True)

In [198]:
agg_df.head(25)

agg_score  count_source  \
Country Name                                                                  
United States of America (the)                       5.990000             1   
China, Hong Kong SAR                                 5.580820             1   
Korea, Republic of                                   5.481020             1   
United States of America                             5.402309             3   
North America                                        5.395205             2   
Hong Kong                                            5.250172             2   
Kosovo                                               5.218340             3   
United States New York City                          5.191101             1   
United States Los Angeles                            5.035912             1   
Republic of Korea (the)                              5.021602             1   
United Kingdom of Great Britain and Northern Ir...   5.015546             1   
Russian Federation Moscow                            4.871741             1   
Hong Kong SAR, China                                 4.865695             4   
Russian Federation Saint Petersburg                  4.834815             1   
Japan Tokyo                                          4.777929             1   
New Zealand                                          4.776443            14   
Japan Osaka                                          4.769446             1   
Mexico Monterrey                                     4.763458             1   
Netherlands (the)                                    4.733822             1   
China Shanghai                                       4.676632             1   
China Beijing                                        4.672141             1   
Brunei Darussalam                                    4.659236             7   
Sweden                                               4.619583            16   
Taiwan, China                                        4.584152             3   
Denmark                                              4.565051            13   

                                                    agg_score_wt  
Country Name                                                      
United States of America (the)                          0.352353  
China, Hong Kong SAR                                    0.328284  
Korea, Republic of                                      0.322413  
United States of America                                0.953349  
North America                                           0.634730  
Hong Kong                                               0.617667  
Kosovo                                                  0.920884  
United States New York City                             0.305359  
United States Los Angeles                               0.296230  
Republic of Korea (the)                                 0.295388  
United Kingdom of Great Britain and Northern Ir...      0.295032  
Russian Federation Moscow                               0.286573  
Hong Kong SAR, China                                    1.144869  
Russian Federation Saint Petersburg                     0.284401  
Japan Tokyo                                             0.281055  
New Zealand                                             3.933541  
Japan Osaka                                             0.280556  
Mexico Monterrey                                        0.280203  
Netherlands (the)                                       0.278460  
China Shanghai                                          0.275096  
China Beijing                                           0.274832  
Brunei Darussalam                                       1.918509  
Sweden                                                  4.347843  
Taiwan, China                                           0.808968  
Denmark                                                 3.490921

In [199]:
agg_df.to_csv('../pillar_scores/business_scores_v0.csv')

In [200]:
### Score Aggregating by Subpillars

In [201]:
df.insert(0,'Pillar','Business')
df

# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()
df['Country Name'] = df['Country Name'].str.strip('**')
df['Country Name'] = df['Country Name'].str.strip('*')

In [202]:
sub_df = df.groupby(['Pillar','Sub-Pillar','Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [203]:
sub_df.columns = ['agg_score', 'count_source' ]

In [204]:
max_number_sources = sub_df.describe()['count_source']['max']

In [205]:
sub_df['agg_score_wt'] = sub_df['agg_score']*(sub_df['count_source']/max_number_sources)

In [206]:
sub_df.to_csv('../subpillar_score/business_scores_subpillar_v0.csv')

### Sources Generation

In [207]:
#Get all countries from Countries.xlsx
countries = pd.read_excel('../../data/Countries.xlsx')
col_names = ['Country or Area']
countries = countries[col_names]
countries.rename(columns = {'Country or Area': 'Country Name'}, inplace = True)

In [208]:
#Get all indicators from names dataframe retrieve at the begining of the script
bnames=bnames[['check','Sub-Pillar','Indicator','Data Source','Data Link']]
bnames.rename(columns = {'check': 'Pillar'}, inplace = True)
bnames = bnames.replace('\n','', regex=True)

In [209]:
#Do a nice cross join so that we have combination of all countries vs all indicators
sources = countries.merge(bnames, how='cross')
sources

Country Name    Pillar           Sub-Pillar  \
0                       Algeria  Business  Technology Adoption   
1                       Algeria  Business  Technology Adoption   
2                       Algeria  Business  Technology Adoption   
3                       Algeria  Business  Technology Adoption   
4                       Algeria  Business  Technology Adoption   
...                         ...       ...                  ...   
4726  Wallis and Futuna Islands  Business  Startup Environment   
4727  Wallis and Futuna Islands  Business  Startup Environment   
4728  Wallis and Futuna Islands  Business  Startup Environment   
4729  Wallis and Futuna Islands  Business  Startup Environment   
4730  Wallis and Futuna Islands  Business  Startup Environment   

                                              Indicator  \
0     UNCTAD Business-to-Consumer (B2C) E-commerce I...   
1         Networking Services (Spend, IT Forecast Data)   
2              Cloud Services (Spend, IT Forecast Data)   
3     ICT task-intensive jobs as a percentage of tot...   
4                       Share of business with internet   
...                                                 ...   
4726                  Ease of finding skilled employees   
4727  Amount invested into startups yearly from priv...   
4728                                 Regulatory Quality   
4729                             Ease of Getting Credit   
4730                            Global Resilience Index   

                                            Data Source  \
0     UNCTAD:  Business-to-Consumer (B2C) E-commerce...   
1          Portulans Institute: Network Readiness Index   
2                                              Statista   
3                           OECD: Going Digital Toolkit   
4              OECD: ICT Access and Usage by Businesses   
...                                                 ...   
4726                                World Bank: GovData   
4727                 OECD: Venture capital investments    
4728  Global Innovation Index/World Bank: Regulatory...   
4729  Global Innovation Index/World Bank: Ease of Ge...   
4730              FM Global: FM Global Resilience Index   

                                              Data Link  
0                                    https://unctad.org  
1                     https://networkreadinessindex.org  
2                              https://www.statista.com  
3                          http://goingdigital.oecd.org  
4                                https://stats.oecd.org  
...                                                 ...  
4726                  https://datacatalog.worldbank.org  
4727                             https://stats.oecd.org  
4728  https://www.globalinnovationindex.org/analysis...  
4729  https://www.globalinnovationindex.org/analysis...  
4730                          https://www.fmglobal.com/  

[4731 rows x 6 columns]

In [210]:
#Make copy of a scores dataframe and add the column available, with value of 1 (string)
#denoting all the country/indicator combinations that have value
dfsources = df[['Country Name','Pillar','Sub-Pillar','Indicator']].copy()
dfsources['Available'] = '1'

In [211]:
#Merge (left join) sources (all country indicator combinations)
#with those in copied scores dataframe
#resulting in a sources dataframe with values of 1 in Available column where there is a value for country/indicator
#and there is a NaN where there's not. 
#subsequently replace NaN with 0 (string)
sources = sources.merge(dfsources, how='left', on=['Country Name','Pillar','Sub-Pillar','Indicator'])
sources['Available'] = sources['Available'].fillna('0')
sources

Country Name    Pillar           Sub-Pillar  \
0                       Algeria  Business  Technology Adoption   
1                       Algeria  Business  Technology Adoption   
2                       Algeria  Business  Technology Adoption   
3                       Algeria  Business  Technology Adoption   
4                       Algeria  Business  Technology Adoption   
...                         ...       ...                  ...   
4726  Wallis and Futuna Islands  Business  Startup Environment   
4727  Wallis and Futuna Islands  Business  Startup Environment   
4728  Wallis and Futuna Islands  Business  Startup Environment   
4729  Wallis and Futuna Islands  Business  Startup Environment   
4730  Wallis and Futuna Islands  Business  Startup Environment   

                                              Indicator  \
0     UNCTAD Business-to-Consumer (B2C) E-commerce I...   
1         Networking Services (Spend, IT Forecast Data)   
2              Cloud Services (Spend, IT Forecast Data)   
3     ICT task-intensive jobs as a percentage of tot...   
4                       Share of business with internet   
...                                                 ...   
4726                  Ease of finding skilled employees   
4727  Amount invested into startups yearly from priv...   
4728                                 Regulatory Quality   
4729                             Ease of Getting Credit   
4730                            Global Resilience Index   

                                            Data Source  \
0     UNCTAD:  Business-to-Consumer (B2C) E-commerce...   
1          Portulans Institute: Network Readiness Index   
2                                              Statista   
3                           OECD: Going Digital Toolkit   
4              OECD: ICT Access and Usage by Businesses   
...                                                 ...   
4726                                World Bank: GovData   
4727                 OECD: Venture capital investments    
4728  Global Innovation Index/World Bank: Regulatory...   
4729  Global Innovation Index/World Bank: Ease of Ge...   
4730              FM Global: FM Global Resilience Index   

                                              Data Link Available  
0                                    https://unctad.org         1  
1                     https://networkreadinessindex.org         1  
2                              https://www.statista.com         0  
3                          http://goingdigital.oecd.org         0  
4                                https://stats.oecd.org         0  
...                                                 ...       ...  
4726                  https://datacatalog.worldbank.org         0  
4727                             https://stats.oecd.org         0  
4728  https://www.globalinnovationindex.org/analysis...         0  
4729  https://www.globalinnovationindex.org/analysis...         0  
4730                          https://www.fmglobal.com/         0  

[4731 rows x 7 columns]

In [212]:
#If sources.csv exists, get the contents, remove everyhing from this pillar, append prepared sources, save csv.
#if sources.csv does not exist, create new file from sources.
from os.path import exists

if exists('../../dashboard/Sources.csv') :
    CurrentSources = pd.read_csv('../../dashboard/Sources.csv', dtype=str)
    CurrentSources = CurrentSources[['Country Name','Pillar','Sub-Pillar','Indicator','Data Source','Data Link','Available']]
    CurrentSources = CurrentSources.loc[CurrentSources['Pillar'] != 'Business']
    CurrentSources = CurrentSources.append(sources)
else :
    CurrentSources = sources
CurrentSources

Country Name       Pillar           Sub-Pillar  \
0                       Algeria  Foundations     Digital Payments   
1                       Algeria  Foundations     Digital Payments   
2                       Algeria  Foundations     Digital Payments   
3                       Algeria  Foundations     Digital Payments   
4                       Algeria  Foundations     Digital Payments   
...                         ...          ...                  ...   
4726  Wallis and Futuna Islands     Business  Startup Environment   
4727  Wallis and Futuna Islands     Business  Startup Environment   
4728  Wallis and Futuna Islands     Business  Startup Environment   
4729  Wallis and Futuna Islands     Business  Startup Environment   
4730  Wallis and Futuna Islands     Business  Startup Environment   

                                              Indicator  \
0                          Digital payments penetration   
1          % of population with digital finance account   
2     Made or received digital payments in the past ...   
3     Made or received digital payments in the past ...   
4     Used a mobile phone or the internet to check a...   
...                                                 ...   
4726                  Ease of finding skilled employees   
4727  Amount invested into startups yearly from priv...   
4728                                 Regulatory Quality   
4729                             Ease of Getting Credit   
4730                            Global Resilience Index   

                                            Data Source  \
0         Portulans Institute: Network Readiness Index    
1                    World Bank: Global Findex database   
2                    World Bank: Global Findex database   
3                    World Bank: Global Findex database   
4                    World Bank: Global Findex database   
...                                                 ...   
4726                                World Bank: GovData   
4727                 OECD: Venture capital investments    
4728  Global Innovation Index/World Bank: Regulatory...   
4729  Global Innovation Index/World Bank: Ease of Ge...   
4730              FM Global: FM Global Resilience Index   

                                              Data Link Available  
0                     https://networkreadinessindex.org         0  
1                     https://datacatalog.worldbank.org         1  
2                     https://datacatalog.worldbank.org         1  
3                     https://datacatalog.worldbank.org         1  
4                     https://datacatalog.worldbank.org         1  
...                                                 ...       ...  
4726                  https://datacatalog.worldbank.org         0  
4727                             https://stats.oecd.org         0  
4728  https://www.globalinnovationindex.org/analysis...         0  
4729  https://www.globalinnovationindex.org/analysis...         0  
4730                          https://www.fmglobal.com/         0  

[32620 rows x 7 columns]

In [213]:
CurrentSources.to_csv('../../dashboard/Sources.csv', index=False)